# Title
[]()

In [1]:
from pandas import json_normalize  
import pandas as pd
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\custom_python")
from silvhua import *
from datetime import datetime
# import json
# from plotly.subplots import make_subplots
# import requests

In [3]:
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")
from response_processing import revive_chatbot, revive_chatbot_dict, save_instance_to_dict
# from wrangling import filter_df_all_conditions
from response_processing import process_chaining_results

In [137]:
# set the option to wrap text within cells
# pd.reset_option('all')
# pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)
pd.set_option('display.width', None)

# Load previous objects

In [10]:
filename = 'batch_Chaining_attributes_2023-05-03_2050.sav'
path = '../text/2023-05-03 4'
chatbots = loadpickle(filename, path)
chatbot_dict = {1: revive_chatbot_dict(chatbots)}

Time completed: 2023-05-03 23:47:14.236824
Dictionary keys: ['text1_prompt00', 'text1_prompt01', 'text1_prompt02', 'text2_prompt00', 'text2_prompt01', 'text2_prompt02']
Article title: The Impact of Sleep Duration on Performance Among Competitive Athletes: A Systematic Literature Review  
	New chatbot attribute added: text
	New chatbot attribute added: system_role
	New chatbot attribute added: temperature
	New chatbot attribute added: max_tokens
	New chatbot attribute added: model
	New chatbot attribute added: qna
		Attribute dictionary keys: ['article_title', 'system_role', 'model', 'text', 'prep step', 'summarization task', 'full summarization task', 'summary']
	New chatbot attribute added: summaries_dict
		Attribute dictionary keys: ['response_01', 'response_02', 'response_03', 'response_04', 'response_05', 'prep_step', 'task', 'prompt']
	New chatbot attribute added: article_title
	New chatbot attribute added: response_regex
	New chatbot attribute added: simple_summary_dict
		Attribu

In [42]:
chatbot_dict[1]['text1_prompt00'].relevance_dict[2]

{'response_01': {0: {'relevance choice': 1,
   'relevance task': 'Rewrite this summary to include a statement of how it is relevant for the audience.         Follow these steps to accomplish this:         \n1. Think about why this might be relevant for the audience in the grand scheme of things.        \n2. If it is not evident why the text is relevant for the audience in the grand scheme of things,         add a sentence to inform the audience. Otherwise, keep it the same.         \n3. Modify the summary if needed to reduce redundancy.         \n4. Check if the content and language are appropriate for the audience.         If it is suitable for the audience, keep it the same. If not, rewrite using terms appropriate for the audience. \\ \n        \n5. Return the final version of the summary to be shown to the audience.         \n6. Remove the backticks.\n        \n\nYour audience consists of',
   'audience': 'people who enjoy sports',
   'full relevance task': 'Rewrite this summary to 

# Attributes

In [68]:
pd.DataFrame(chatbot_dict[1]['text1_prompt00'].qna).reset_index(names=['choice'])

,choice,article_title,system_role,model,text,prep step,summarization task,full summarization task,summary
0,0,The Impact of Sleep Duration on Performance Among Competitive Athletes: A Systematic Literature ...,You are an expert at science communication.,gpt-3.5-turbo,The Impact of Sleep Duration on Performance Among Competitive Athletes: A Systematic Literature ...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post.,Take the key points and numerical descriptors to Summarize for a LinkedIn post.,A systematic literature review found no negative association between sleep duration and athletic...
1,1,The Impact of Sleep Duration on Performance Among Competitive Athletes: A Systematic Literature ...,You are an expert at science communication.,gpt-3.5-turbo,The Impact of Sleep Duration on Performance Among Competitive Athletes: A Systematic Literature ...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post.,Take the key points and numerical descriptors to Summarize for a LinkedIn post.,"""Sleep duration has a positive impact on athletic performance, according to a systematic literat..."
2,2,The Impact of Sleep Duration on Performance Among Competitive Athletes: A Systematic Literature ...,You are an expert at science communication.,gpt-3.5-turbo,The Impact of Sleep Duration on Performance Among Competitive Athletes: A Systematic Literature ...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post.,Take the key points and numerical descriptors to Summarize for a LinkedIn post.,"""New literature review shows that increased sleep duration has a positive effect on athletic per..."
3,3,The Impact of Sleep Duration on Performance Among Competitive Athletes: A Systematic Literature ...,You are an expert at science communication.,gpt-3.5-turbo,The Impact of Sleep Duration on Performance Among Competitive Athletes: A Systematic Literature ...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post.,Take the key points and numerical descriptors to Summarize for a LinkedIn post.,"""Sleep duration has a positive impact on athletic performance among competitive athletes, accord..."
4,4,The Impact of Sleep Duration on Performance Among Competitive Athletes: A Systematic Literature ...,You are an expert at science communication.,gpt-3.5-turbo,The Impact of Sleep Duration on Performance Among Competitive Athletes: A Systematic Literature ...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post.,Take the key points and numerical descriptors to Summarize for a LinkedIn post.,"""Sleep duration has a positive effect on athletic performance, according to a systematic literat..."


# Update function

In [52]:
import string
def merge_chaining_results(
        chain_results_dict, chatbot_dict, iteration_id, 
        empty_columns=None, pivot=True, validate=None,
        chatbot_id=None, save_df=False, save_chatbot=False, 
        csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
        pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles',
        json_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\json'
        ):
    """
    Create a dataframe of new 'simple' or 'relevance' summaries from a Chaining object.
    Merge it with the original summaries DataFrame.

    Parameters:
        - chain_results_dict (dict): dictionary of DataFrames.
        - qna_dict (dict): dictionary of QnA DataFrames.
        - chatbot_dict (dict): dictionary of Chaining objects.
        - iteration_id (int, float, or string): iteration_id (dict key) of the chatbot_dict to process.
        - results_type (str): 'simple' or 'relevance'. Default is 'simple'.
        - empty_columns (Bool, int, or dict): dictionary of empty columns to add to the DataFrame. 
            If True or 1, default dictionary is used.
            If False or 0, no empty columns are added.
        - pivot (Bool): whether to pivot the relevance summaries DataFrame. Default is True.
        - validate (str): Argument to pass to pd.merge() to validate the merge.
        - chatbot_id (int, float, or string): chatbot_id (dict key) of the chatbot_dict to process.
        - save_df, save_chatbot (Bool): whether to save the DataFrame and chatbot_dict.

    """
    df_list_simple = []
    df_list_relevance = []
    qna_dfs_list = []
    iteration_id = chatbot_id if chatbot_id != None else iteration_id
    for chatbot_key in chatbot_dict[iteration_id].keys():
        print(f'Processing {chatbot_key}...')
        try: 
            n_previous_prompts_simple = chatbot_dict[iteration_id][chatbot_key].previous_n_prompts['simple']
            print(f'\tNumber of previous relevance prompts: {n_previous_prompts_simple}')
        except:
            n_previous_prompts_simple = 0
            print(f'No previous simple prompts for {chatbot_key}')
        try: 
            n_previous_prompts_relevance = chatbot_dict[iteration_id][chatbot_key].previous_n_prompts['relevance']
            print(f'\tNumber of previous relevance prompts: {n_previous_prompts_relevance}')
        except:
            n_previous_prompts_relevance = 0
            print(f'No previous relevance prompts for {chatbot_key}')
            
        qna_dfs_list.append(pd.DataFrame(chatbot_dict[iteration_id][chatbot_key].qna))

        total_n_prompts_simple = len(chatbot_dict[iteration_id][chatbot_key].simple_summary_dict)
        results_dict_simple = dict()
        for prompt_number in range(n_previous_prompts_simple+1, total_n_prompts_simple+1):
            results_dict_simple[prompt_number] = chatbot_dict[iteration_id][chatbot_key].simple_summary_dict[prompt_number]
        chatbot_dict[iteration_id][chatbot_key].simple_summary_dict

        total_n_prompts_relevance = len(chatbot_dict[iteration_id][chatbot_key].relevance_dict)
        results_dict_relevance = dict()
        for prompt_number_relevance in range(n_previous_prompts_relevance+1, total_n_prompts_relevance+1):
            # print(f'\tAppending results for prompt {prompt_number_relevance} of {total_n_prompts_relevance}')
            results_dict_relevance[prompt_number_relevance] = chatbot_dict[iteration_id][chatbot_key].relevance_dict[prompt_number_relevance]
        for iteration_key_simple, iteration_key_relevance in zip(results_dict_simple.keys(), results_dict_relevance.keys()):
            response_keys_simple = sorted([text_prompt_key for text_prompt_key in results_dict_simple[iteration_key_simple].keys()])
            response_keys_relevance = sorted([text_prompt_key for text_prompt_key in results_dict_relevance[iteration_key_relevance].keys()])
            # print(f'\tAppending results for {iteration_key_simple}: ', end='')
            for response_key_simple in response_keys_simple:
                df_list_simple.append(pd.DataFrame(results_dict_simple[iteration_key_simple][response_key_simple]).transpose())
                # print(f'{response_key_simple}, ', end='')
            print('')
            for response_key_relevance in response_keys_relevance:
                df_list_relevance.append(pd.DataFrame(results_dict_relevance[iteration_key_relevance][response_key_relevance]).transpose())
                # print(f'{response_key_relevance}, ', end='')
            print('')

    
    simple_summary_df = pd.concat(df_list_simple)
    relevance_df = pd.concat(df_list_relevance)
    qna_df = pd.concat(qna_dfs_list)
    print(f'Original summaries DataFrame shape: {qna_df.shape}')
    print(f'Original summaries Dataframe columns: {qna_df.columns}')
    print('Simple summaries DataFrame shape:', simple_summary_df.shape)
    print(f'\tSimple summaries DataFrame columns: {[col for col in simple_summary_df.columns]}')
    print('Relevance summaries DataFrame shape:', relevance_df.shape)
    print(f'\tRelevance summaries DataFrame columns: {[col for col in relevance_df.columns]}')
    print(f'Unique relevance audience values: {relevance_df.audience.unique()}')
    
    original_summary_columns = [
        'article_title',
        # 'choice',
        'system_role',
        'model',
        'text',
        'prep step',
        'summarization task',
        'full summarization task',
        'summary',
    ]
    
    if pivot == False:
        validate=None
        common_columns = list(set(simple_summary_df.columns) & set(relevance_df.columns))
        common_columns.remove('audience')
        new_results = qna_df.merge(
            simple_summary_df, how='outer',
            right_on='original summary',
            left_on='summary',
            validate=validate
            ).drop(columns='original summary')
        
        print(f'DataFrame shape after merging with simple summaries: {new_results.shape}')
        new_results= new_results.merge(
            relevance_df, how='outer', suffixes=(' simplify', ' relevance'),
            left_on='summary', right_on='preceding summary', validate=validate
            ).drop(columns='preceding summary')
    else:
        relevance_pivot_df = relevance_df.pivot(
            columns=['audience'],
            values='relevance statement',
            index=['preceding summary', 'relevance task']
        ).reset_index()
        new_results = simple_summary_df.merge(
            relevance_pivot_df, how='outer', suffixes=(' simplify', ' relevance'),
            on=['summary'],
            validate='m:1' if validate else None
        )

    if empty_columns:
        if pivot == False:
            if (type(empty_columns) != dict):
                if results_type=='simple':
                    empty_columns = {
                        "choice numnber": "C",
                        "original summary content rating": "K",
                        "original summary language rating": "L",
                        "top summary": "M",
                    }
                else:
                    empty_columns = {
                        "choice numnber": "C",
                        "original summary content rating": "K",
                        "original summary language rating": "L",
                        "top summary": "M",
                        "simplify audience": "O",
                        "simplify task": "P",
                        "full simplify task": "Q",
                        "simple summary": "R",
                        "simple summary content rating": "S",
                        "simple summary language rating": "T",
                        "top simple summary": "U",
                    }
        else:           
            if (type(empty_columns) != dict):
                empty_columns = {
                    "choice numnber": "C",
                        "original summary content rating": "K",
                    "original summary language rating": "L",
                    "top summary": "M",
                    "simple summary choice": "N",
                    "simplify audience": "O",
                    "simplify task": "P",
                    "full simplify task": "Q",
                    "simple summary": "R",
                    "simple summary content rating": "S",
                    "simple summary language rating": "T",
                    'top simple summary': 'u',
                    'full add relevance task': 'w',
                    'added relevance content rating': 'y',
                    'added relevance language rating': 'z',
                    'top added relevance': 'aa',
                    'add relevance task (seniors)': 'AB',
                    'full add relevance task (seniors)': 'AC',
                }
        print(f'Merged DataFrame shape: {new_results.shape}')
        print('\nColumns before adding empty columns:', [column for column in new_results.columns])
        print('Inserting empty columns...', end='\n\t')
        spreadsheet_columns = [letter for letter in string.ascii_uppercase]+['A'+letter for letter in string.ascii_uppercase]
        alphabet_dict = {char:idx for idx, char in enumerate(spreadsheet_columns)}
        for column_name, column_number in empty_columns.items():
            empty_column_loc = alphabet_dict[empty_columns[column_name].upper()] -1
            new_results.insert(loc=empty_column_loc, column=column_name, value='')
            print(f'{empty_columns[column_name].upper()} ({empty_column_loc}): {column_name}', end=', ')
        new_results.columns = [
            f'{spreadsheet_columns[index+1]}: {column}' for index, column in enumerate(new_results.columns)
            ]

    print(f'\n** Merged dataframe shape:', new_results.shape)
    print([column for column in new_results.columns])
    chain_results_dict[iteration_id] = new_results
    try:
        original_summary_time = next(iter(chatbot_dict[iteration_id].values())).date_created
    except:
        original_summary_time = 'previous_summaries'
    print(f'Original summary time: {original_summary_time}')
    if save_df:
        try:
            save_output(
                chain_results_dict[iteration_id], description=f'batch_Chaining_attributes_{original_summary_time}_append_{results_type}',
                csv_path=csv_path, pickle_path=pickle_path)
            print('')
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} DataFrame')
    if save_chatbot:
        try:
            print('Saving Chaining object (chatbot)...')
            save_instance_to_dict(
                chatbot_dict[iteration_id], 
                description=f'batch_Chaining_attributes_{original_summary_time}_append_{results_type}',
                pickle_path=pickle_path, json_path=json_path
                )
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} chatbot')
            
    return chain_results_dict

chain_results_dict = dict()
iteration_id = 1

chain_results_dict = merge_chaining_results(
    chain_results_dict, chatbot_dict, iteration_id, pivot=False
    )
chain_results_dict[iteration_id].filter(regex='article|summary|statement|full')


Processing text1_prompt00...
No previous simple prompts for text1_prompt00
No previous relevance prompts for text1_prompt00


Processing text1_prompt01...
No previous simple prompts for text1_prompt01
No previous relevance prompts for text1_prompt01


Processing text1_prompt02...
No previous simple prompts for text1_prompt02
No previous relevance prompts for text1_prompt02


Processing text2_prompt00...
No previous simple prompts for text2_prompt00
No previous relevance prompts for text2_prompt00


Processing text2_prompt01...
No previous simple prompts for text2_prompt01
No previous relevance prompts for text2_prompt01


Processing text2_prompt02...
No previous simple prompts for text2_prompt02
No previous relevance prompts for text2_prompt02


Original summaries DataFrame shape: (30, 8)
Original summaries Dataframe columns: Index(['article_title', 'system_role', 'model', 'text', 'prep step',
       'summarization task', 'full summarization task', 'summary'],
      dtype='object')
Sim

,article_title,full summarization task,summary,full simplify task,simple summary,simple summary choice,full relevance task,relevance statement
0,The Impact of Sleep Duration on Performance Among Competitive Athletes: A Systematic Literature Review,Take the key points and numerical descriptors to Summarize for a LinkedIn post.,"A systematic literature review found no negative association between sleep duration and athletic performance. Aerobic sports are more sensitive to sleep duration compared to anaerobic activity. Long-term sleep extension interventions improved tactical and technical skills. Short-term sleep deprivation may be more harmful than sleep extension, but more research is needed to assess potential adverse effects of chronic sleep loss on athletic performance. Strategies to improve sleep before and leading up to important sports events are likely to improve performance. #sleep #athleticperformance #sports #research","If needed, rewrite the text using terms appropriate for the audience. If not keep it the same. Follow these steps to accomplish this: \n1. Check if the content and language are appropriate for the audience. \n2. If it is suitable for the audience, keep it the same. If not, rewrite using terms appropriate for the audience. \ \n \n3. Return the final version of the summary to be shown to the audience. \n\nYour audience is for people without a science background","A recent study found that getting enough sleep doesn't negatively affect how well athletes perform. However, sports that require a lot of endurance are more affected by sleep than sports that require short bursts of energy. Sleeping more over a long period of time can help improve an athlete's skills. But not getting enough sleep in the short term can be worse than getting more sleep. We need more research to know how not getting enough sleep over a long time affects athletes. If athletes get enough sleep before important events, they are more likely to do well. #sleep #athleticperformance #sports #research",1,"Rewrite this summary to include a statement of how it is relevant for the audience. Follow these steps to accomplish this: \n1. Think about why this might be relevant for the audience in the grand scheme of things. \n2. If it is not evident why the text is relevant for the audience in the grand scheme of things, add a sentence to inform the audience. Otherwise, keep it the same. \n3. Modify the summary if needed to reduce redundancy. \n4. Check if the content and language are appropriate for the audience. If it is suitable for the audience, keep it the same. If not, rewrite using terms appropriate for the audience. \ \n \n5. Return the final version of the summary to be shown to the audience. \n6. Remove the backticks.\n \n\nYour audience consists of seniors","A recent study found that getting enough sleep is important for athletic performance, especially for aerobic sports. Interestingly, long-term sleep extension interventions can even improve tactical and technical skills. However, short-term sleep deprivation can be more harmful than sleep extension. While more research is needed to fully understand the effects of chronic sleep loss on athletic performance, it is clear that getting enough sleep before important sports events can improve performance. This information is relevant for seniors who are interested in staying active and maintaining their athletic abilities. #sleep #athleticperformance #sports #research"
1,The Impact of Sleep Duration on Performance Among Competitive Athletes: A Systematic Literature Review,Take the key points and numerical descriptors to Summarize for a LinkedIn post.,"""Sleep duration has a positive impact on athletic performance, according to a systematic literature review of published studies. None found a negative association between sleep duration and performance outcomes. Aerobic activity is more sensitive to sleep duration compared to anaerobic activity. Studies on outcomes requiring tactical and/or

## Iteration 2

In [59]:
import string
def merge_chaining_results(
        chain_results_dict, chatbot_dict, iteration_id, 
        empty_columns=None, pivot=True, validate=None,
        chatbot_id=None, save_df=False, save_chatbot=False, 
        csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
        pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles',
        json_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\json'
        ):
    """
    Create a dataframe of new 'simple' or 'relevance' summaries from a Chaining object.
    Merge it with the original summaries DataFrame.

    Parameters:
        - chain_results_dict (dict): dictionary of DataFrames.
        - qna_dict (dict): dictionary of QnA DataFrames.
        - chatbot_dict (dict): dictionary of Chaining objects.
        - iteration_id (int, float, or string): iteration_id (dict key) of the chatbot_dict to process.
        - results_type (str): 'simple' or 'relevance'. Default is 'simple'.
        - empty_columns (Bool, int, or dict): dictionary of empty columns to add to the DataFrame. 
            If True or 1, default dictionary is used.
            If False or 0, no empty columns are added.
        - pivot (Bool): whether to pivot the relevance summaries DataFrame. Default is True.
        - validate (str): Argument to pass to pd.merge() to validate the merge.
        - chatbot_id (int, float, or string): chatbot_id (dict key) of the chatbot_dict to process.
        - save_df, save_chatbot (Bool): whether to save the DataFrame and chatbot_dict.

    """
    df_list_simple = []
    df_list_relevance = []
    qna_dfs_list = []
    iteration_id = chatbot_id if chatbot_id != None else iteration_id
    for chatbot_key in chatbot_dict[iteration_id].keys():
        print(f'Processing {chatbot_key}...')
        try: 
            n_previous_prompts_simple = chatbot_dict[iteration_id][chatbot_key].previous_n_prompts['simple']
            print(f'\tNumber of previous relevance prompts: {n_previous_prompts_simple}')
        except:
            n_previous_prompts_simple = 0
            print(f'No previous simple prompts for {chatbot_key}')
        try: 
            n_previous_prompts_relevance = chatbot_dict[iteration_id][chatbot_key].previous_n_prompts['relevance']
            print(f'\tNumber of previous relevance prompts: {n_previous_prompts_relevance}')
        except:
            n_previous_prompts_relevance = 0
            print(f'No previous relevance prompts for {chatbot_key}')
            
        qna_dfs_list.append(pd.DataFrame(chatbot_dict[iteration_id][chatbot_key].qna))

        # create results dictionaries that only grabs the results of the new prompts instead of all
        results_dict_simple = dict()
        total_n_prompts_simple = len(chatbot_dict[iteration_id][chatbot_key].simple_summary_dict)
        for prompt_number in range(n_previous_prompts_simple+1, total_n_prompts_simple+1):
            results_dict_simple[prompt_number] = chatbot_dict[iteration_id][chatbot_key].simple_summary_dict[prompt_number]
        chatbot_dict[iteration_id][chatbot_key].simple_summary_dict

        results_dict_relevance = dict()
        total_n_prompts_relevance = len(chatbot_dict[iteration_id][chatbot_key].relevance_dict)
        for prompt_number_relevance in range(n_previous_prompts_relevance+1, total_n_prompts_relevance+1):
            # print(f'\tAppending results for prompt {prompt_number_relevance} of {total_n_prompts_relevance}')
            results_dict_relevance[prompt_number_relevance] = chatbot_dict[iteration_id][chatbot_key].relevance_dict[prompt_number_relevance]

        for iteration_key_simple in results_dict_simple.keys():
            response_keys_simple = sorted([text_prompt_key for text_prompt_key in results_dict_simple[iteration_key_simple].keys()])
            # print(f'\tAppending results for {iteration_key_simple}: ', end='')

            for response_key_simple in response_keys_simple:
                df_list_simple.append(pd.DataFrame(results_dict_simple[iteration_key_simple][response_key_simple]).transpose())
                # print(f'{response_key_simple}, ', end='')
            print('')
        for iteration_key_relevance in results_dict_relevance.keys():
            response_keys_relevance = sorted([text_prompt_key for text_prompt_key in results_dict_relevance[iteration_key_relevance].keys()])
            for response_key_relevance in response_keys_relevance:
                df_list_relevance.append(pd.DataFrame(results_dict_relevance[iteration_key_relevance][response_key_relevance]).transpose())
                # print(f'{response_key_relevance}, ', end='')
            print('')

    
    simple_summary_df = pd.concat(df_list_simple)
    relevance_df = pd.concat(df_list_relevance)
    qna_df = pd.concat(qna_dfs_list)
    print(f'Original summaries DataFrame shape: {qna_df.shape}')
    print(f'Original summaries Dataframe columns: {qna_df.columns}')
    print('Simple summaries DataFrame shape:', simple_summary_df.shape)
    print(f'\tSimple summaries DataFrame columns: {[col for col in simple_summary_df.columns]}')
    print('Relevance summaries DataFrame shape:', relevance_df.shape)
    print(f'\tRelevance summaries DataFrame columns: {[col for col in relevance_df.columns]}')
    print(f'Unique relevance audience values: {relevance_df.audience.unique()}')
    
    original_summary_columns = [
        'article_title',
        # 'choice',
        'system_role',
        'model',
        'text',
        'prep step',
        'summarization task',
        'full summarization task',
        'summary',
    ]
    
    if pivot == False:
        validate=None
        common_columns = list(set(simple_summary_df.columns) & set(relevance_df.columns))
        common_columns.remove('audience')
        new_results = qna_df.merge(
            simple_summary_df, how='outer',
            right_on='original summary',
            left_on='summary',
            validate=validate
            ).drop(columns='original summary')
        
        print(f'DataFrame shape after merging with simple summaries: {new_results.shape}')
        new_results= new_results.merge(
            relevance_df, how='outer', suffixes=(' simplify', ' relevance'),
            left_on='summary', right_on='preceding summary', validate=validate
            ).drop(columns='preceding summary')
    else:
        relevance_pivot_df = relevance_df.pivot(
            columns=['audience'],
            values='relevance statement',
            index=['preceding summary', 'relevance task']
        ).reset_index()
        new_results = simple_summary_df.merge(
            relevance_pivot_df, how='outer', suffixes=(' simplify', ' relevance'),
            on=['summary'],
            validate='m:1' if validate else None
        )

    if empty_columns:
        if pivot == False:
            if (type(empty_columns) != dict):
                if results_type=='simple':
                    empty_columns = {
                        "choice numnber": "C",
                        "original summary content rating": "K",
                        "original summary language rating": "L",
                        "top summary": "M",
                    }
                else:
                    empty_columns = {
                        "choice numnber": "C",
                        "original summary content rating": "K",
                        "original summary language rating": "L",
                        "top summary": "M",
                        "simplify audience": "O",
                        "simplify task": "P",
                        "full simplify task": "Q",
                        "simple summary": "R",
                        "simple summary content rating": "S",
                        "simple summary language rating": "T",
                        "top simple summary": "U",
                    }
        else:           
            if (type(empty_columns) != dict):
                empty_columns = {
                    "choice numnber": "C",
                        "original summary content rating": "K",
                    "original summary language rating": "L",
                    "top summary": "M",
                    "simple summary choice": "N",
                    "simplify audience": "O",
                    "simplify task": "P",
                    "full simplify task": "Q",
                    "simple summary": "R",
                    "simple summary content rating": "S",
                    "simple summary language rating": "T",
                    'top simple summary': 'u',
                    'full add relevance task': 'w',
                    'added relevance content rating': 'y',
                    'added relevance language rating': 'z',
                    'top added relevance': 'aa',
                    'add relevance task (seniors)': 'AB',
                    'full add relevance task (seniors)': 'AC',
                }
        print(f'Merged DataFrame shape: {new_results.shape}')
        print('\nColumns before adding empty columns:', [column for column in new_results.columns])
        print('Inserting empty columns...', end='\n\t')
        spreadsheet_columns = [letter for letter in string.ascii_uppercase]+['A'+letter for letter in string.ascii_uppercase]
        alphabet_dict = {char:idx for idx, char in enumerate(spreadsheet_columns)}
        for column_name, column_number in empty_columns.items():
            empty_column_loc = alphabet_dict[empty_columns[column_name].upper()] -1
            new_results.insert(loc=empty_column_loc, column=column_name, value='')
            print(f'{empty_columns[column_name].upper()} ({empty_column_loc}): {column_name}', end=', ')
        new_results.columns = [
            f'{spreadsheet_columns[index+1]}: {column}' for index, column in enumerate(new_results.columns)
            ]

    print(f'\n** Merged dataframe shape:', new_results.shape)
    print([column for column in new_results.columns])
    chain_results_dict[iteration_id] = new_results
    try:
        original_summary_time = next(iter(chatbot_dict[iteration_id].values())).date_created
    except:
        original_summary_time = 'previous_summaries'
    print(f'Original summary time: {original_summary_time}')
    if save_df:
        try:
            save_output(
                chain_results_dict[iteration_id], description=f'batch_Chaining_attributes_{original_summary_time}_append_{results_type}',
                csv_path=csv_path, pickle_path=pickle_path)
            print('')
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} DataFrame')
    if save_chatbot:
        try:
            print('Saving Chaining object (chatbot)...')
            save_instance_to_dict(
                chatbot_dict[iteration_id], 
                description=f'batch_Chaining_attributes_{original_summary_time}_append_{results_type}',
                pickle_path=pickle_path, json_path=json_path
                )
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} chatbot')
            
    return chain_results_dict

chain_results_dict = dict()
iteration_id = 1

chain_results_dict = merge_chaining_results(
    chain_results_dict, chatbot_dict, iteration_id, pivot=False
    )
# chain_results_dict[iteration_id].filter(regex='article|summary|statement|full')


Processing text1_prompt00...
No previous simple prompts for text1_prompt00
No previous relevance prompts for text1_prompt00



Processing text1_prompt01...
No previous simple prompts for text1_prompt01
No previous relevance prompts for text1_prompt01



Processing text1_prompt02...
No previous simple prompts for text1_prompt02
No previous relevance prompts for text1_prompt02



Processing text2_prompt00...
No previous simple prompts for text2_prompt00
No previous relevance prompts for text2_prompt00



Processing text2_prompt01...
No previous simple prompts for text2_prompt01
No previous relevance prompts for text2_prompt01



Processing text2_prompt02...
No previous simple prompts for text2_prompt02
No previous relevance prompts for text2_prompt02



Original summaries DataFrame shape: (30, 8)
Original summaries Dataframe columns: Index(['article_title', 'system_role', 'model', 'text', 'prep step',
       'summarization task', 'full summarization task', 'summary'],
      dtype='object

In [63]:
chain_results_dict[iteration_id]

,article_title,system_role,model,text,prep step,summarization task,full summarization task,summary,audience simplify,full simplify task,simple summary,simple summary choice,simplify task,audience relevance,full relevance task,relevance choice,relevance statement,relevance task
0,The Impact of Sleep Duration on Performance Among Competitive Athletes: A Systematic Literature ...,You are an expert at science communication.,gpt-3.5-turbo,The Impact of Sleep Duration on Performance Among Competitive Athletes: A Systematic Literature ...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post.,Take the key points and numerical descriptors to Summarize for a LinkedIn post.,A systematic literature review found no negative association between sleep duration and athletic...,people without a science background,"If needed, rewrite the text using terms appropriate for the audience. If not keep it the same. ...",A recent study found that getting enough sleep doesn't negatively affect how well athletes perfo...,1,"If needed, rewrite the text using terms appropriate for the audience. If not keep it the same. ...",seniors,Rewrite this summary to include a statement of how it is relevant for the audience. Foll...,1,"A recent study found that getting enough sleep is important for athletic performance, especially...",Rewrite this summary to include a statement of how it is relevant for the audience. Foll...
1,The Impact of Sleep Duration on Performance Among Competitive Athletes: A Systematic Literature ...,You are an expert at science communication.,gpt-3.5-turbo,The Impact of Sleep Duration on Performance Among Competitive Athletes: A Systematic Literature ...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post.,Take the key points and numerical descriptors to Summarize for a LinkedIn post.,A systematic literature review found no negative association between sleep duration and athletic...,people without a science background,"If needed, rewrite the text using terms appropriate for the audience. If not keep it the same. ...",A recent study found that getting enough sleep doesn't negatively affect how well athletes perfo...,1,"If needed, rewrite the text using terms appropriate for the audience. If not keep it the same. ...",people who enjoy sports,Rewrite this summary to include a statement of how it is relevant for the audience. Foll...,1,"Getting enough sleep is important for athletic performance, especially for aerobic sports. A rev...",Rewrite this summary to include a statement of how it is relevant for the audience. Foll...
2,The Impact of Sleep Duration on Performance Among Competitive Athletes: A Systematic Literature ...,You are an expert at science communication.,gpt-3.5-turbo,The Impact of Sleep Duration on Performance Among Competitive Athletes: A Systematic Literature ...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post.,Take the key points and numerical descriptors to Summarize for a LinkedIn post.,"""Sleep duration has a positive impact on athletic performance, according to a systematic literat...",people without a science background,"If needed, rewrite the text using terms appropriate for the audience. If not keep it the same. ...","""Getting enough sleep can help you perform better in sports, according to a review of many studi...",1,"If needed, rewrite the text using terms appropriate for the audience. If not keep it the same. ...",seniors,Rewrite this summary to include a statement of how it is relevant for the audience. Foll...,1,"Sleeping well can help seniors improve their athletic performance, according to a review of publ...",Rewrite this summary to include a statement of how it is relevant for the audience. Foll...
3,The Impact of Sleep Duration on Performance Among Competitive Athletes: A Systematic Literature ...,You are an expert at science communication.,gpt-3.5-turbo,The Impact of Sleep Duration on Performance Among Competitive Athletes: 

## iteration 3 change to right merge like in `process_chaining_results`; add choice # column

In [75]:
import string
def merge_chaining_results(
        chain_results_dict, chatbot_dict, iteration_id, 
        empty_columns=None, pivot=True, validate=None,
        chatbot_id=None, save_df=False, save_chatbot=False, 
        csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
        pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles',
        json_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\json'
        ):
    """
    Create a dataframe of new 'simple' or 'relevance' summaries from a Chaining object.
    Merge it with the original summaries DataFrame.

    Parameters:
        - chain_results_dict (dict): dictionary of DataFrames.
        - qna_dict (dict): dictionary of QnA DataFrames.
        - chatbot_dict (dict): dictionary of Chaining objects.
        - iteration_id (int, float, or string): iteration_id (dict key) of the chatbot_dict to process.
        - results_type (str): 'simple' or 'relevance'. Default is 'simple'.
        - empty_columns (Bool, int, or dict): dictionary of empty columns to add to the DataFrame. 
            If True or 1, default dictionary is used.
            If False or 0, no empty columns are added.
        - pivot (Bool): whether to pivot the relevance summaries DataFrame. Default is True.
        - validate (str): Argument to pass to pd.merge() to validate the merge.
        - chatbot_id (int, float, or string): chatbot_id (dict key) of the chatbot_dict to process.
        - save_df, save_chatbot (Bool): whether to save the DataFrame and chatbot_dict.

    """
    df_list_simple = []
    df_list_relevance = []
    qna_dfs_list = []
    iteration_id = chatbot_id if chatbot_id != None else iteration_id
    for chatbot_key in chatbot_dict[iteration_id].keys():
        print(f'Processing {chatbot_key}...')
        try: 
            n_previous_prompts_simple = chatbot_dict[iteration_id][chatbot_key].previous_n_prompts['simple']
            print(f'\tNumber of previous relevance prompts: {n_previous_prompts_simple}')
        except:
            n_previous_prompts_simple = 0
            print(f'No previous simple prompts for {chatbot_key}')
        try: 
            n_previous_prompts_relevance = chatbot_dict[iteration_id][chatbot_key].previous_n_prompts['relevance']
            print(f'\tNumber of previous relevance prompts: {n_previous_prompts_relevance}')
        except:
            n_previous_prompts_relevance = 0
            print(f'No previous relevance prompts for {chatbot_key}')
            
        # qna_dfs_list.append(pd.DataFrame(chatbot_dict[iteration_id][chatbot_key].qna))
        # qna_dfs_list.append(pd.DataFrame(chatbot_dict[iteration_id][chatbot_key].qna).reset_index(inplace=True, names=['choice']))
        qna_dfs_list.append(pd.DataFrame(chatbot_dict[iteration_id][chatbot_key].qna).reset_index(names=['choice']))

        # create results dictionaries that only grabs the results of the new prompts instead of all
        results_dict_simple = dict()
        total_n_prompts_simple = len(chatbot_dict[iteration_id][chatbot_key].simple_summary_dict)
        for prompt_number in range(n_previous_prompts_simple+1, total_n_prompts_simple+1):
            results_dict_simple[prompt_number] = chatbot_dict[iteration_id][chatbot_key].simple_summary_dict[prompt_number]
        chatbot_dict[iteration_id][chatbot_key].simple_summary_dict

        results_dict_relevance = dict()
        total_n_prompts_relevance = len(chatbot_dict[iteration_id][chatbot_key].relevance_dict)
        for prompt_number_relevance in range(n_previous_prompts_relevance+1, total_n_prompts_relevance+1):
            # print(f'\tAppending results for prompt {prompt_number_relevance} of {total_n_prompts_relevance}')
            results_dict_relevance[prompt_number_relevance] = chatbot_dict[iteration_id][chatbot_key].relevance_dict[prompt_number_relevance]

        for iteration_key_simple in results_dict_simple.keys():
            response_keys_simple = sorted([text_prompt_key for text_prompt_key in results_dict_simple[iteration_key_simple].keys()])
            # print(f'\tAppending results for {iteration_key_simple}: ', end='')

            for response_key_simple in response_keys_simple:
                df_list_simple.append(pd.DataFrame(results_dict_simple[iteration_key_simple][response_key_simple]).transpose())
                # print(f'{response_key_simple}, ', end='')
            print('')
        for iteration_key_relevance in results_dict_relevance.keys():
            response_keys_relevance = sorted([text_prompt_key for text_prompt_key in results_dict_relevance[iteration_key_relevance].keys()])
            for response_key_relevance in response_keys_relevance:
                df_list_relevance.append(pd.DataFrame(results_dict_relevance[iteration_key_relevance][response_key_relevance]).transpose())
                # print(f'{response_key_relevance}, ', end='')
            print('')

    
    simple_summary_df = pd.concat(df_list_simple)
    relevance_df = pd.concat(df_list_relevance)
    qna_df = pd.concat(qna_dfs_list)
    print(f'Original summaries DataFrame shape: {qna_df.shape}')
    print(f'Original summaries Dataframe columns: {qna_df.columns}')
    print('Simple summaries DataFrame shape:', simple_summary_df.shape)
    print(f'\tSimple summaries DataFrame columns: {[col for col in simple_summary_df.columns]}')
    print('Relevance summaries DataFrame shape:', relevance_df.shape)
    print(f'\tRelevance summaries DataFrame columns: {[col for col in relevance_df.columns]}')
    print(f'Unique relevance audience values: {relevance_df.audience.unique()}')
    
    original_summary_columns = [
        'article_title',
        # 'choice',
        'system_role',
        'model',
        'text',
        'prep step',
        'summarization task',
        'full summarization task',
        'summary',
    ]
    
    if pivot == False:
        validate=None
        common_columns = list(set(simple_summary_df.columns) & set(relevance_df.columns))
        common_columns.remove('audience')
        new_results = qna_df.merge(
            simple_summary_df, how='right',
            right_on='original summary',
            left_on='summary',
            validate=validate
            ).drop(columns='original summary')
        
        print(f'DataFrame shape after merging with simple summaries: {new_results.shape}')
        new_results= new_results.merge(
            relevance_df, how='outer', suffixes=(' simplify', ' relevance'),
            left_on='summary', right_on='preceding summary', validate=validate
            ).drop(columns='preceding summary')
    else:
        relevance_pivot_df = relevance_df.pivot(
            columns=['audience'],
            values='relevance statement',
            index=['preceding summary', 'relevance task']
        ).reset_index()
        new_results = simple_summary_df.merge(
            relevance_pivot_df, how='outer', suffixes=(' simplify', ' relevance'),
            on=['summary'],
            validate='m:1' if validate else None
        )

    if empty_columns:
        if pivot == False:
            if (type(empty_columns) != dict):
                if results_type=='simple':
                    empty_columns = {
                        "choice numnber": "C",
                        "original summary content rating": "K",
                        "original summary language rating": "L",
                        "top summary": "M",
                    }
                else:
                    empty_columns = {
                        "choice numnber": "C",
                        "original summary content rating": "K",
                        "original summary language rating": "L",
                        "top summary": "M",
                        "simplify audience": "O",
                        "simplify task": "P",
                        "full simplify task": "Q",
                        "simple summary": "R",
                        "simple summary content rating": "S",
                        "simple summary language rating": "T",
                        "top simple summary": "U",
                    }
        else:           
            if (type(empty_columns) != dict):
                empty_columns = {
                    "choice numnber": "C",
                        "original summary content rating": "K",
                    "original summary language rating": "L",
                    "top summary": "M",
                    "simple summary choice": "N",
                    "simplify audience": "O",
                    "simplify task": "P",
                    "full simplify task": "Q",
                    "simple summary": "R",
                    "simple summary content rating": "S",
                    "simple summary language rating": "T",
                    'top simple summary': 'u',
                    'full add relevance task': 'w',
                    'added relevance content rating': 'y',
                    'added relevance language rating': 'z',
                    'top added relevance': 'aa',
                    'add relevance task (seniors)': 'AB',
                    'full add relevance task (seniors)': 'AC',
                }
        print(f'Merged DataFrame shape: {new_results.shape}')
        print('\nColumns before adding empty columns:', [column for column in new_results.columns])
        print('Inserting empty columns...', end='\n\t')
        spreadsheet_columns = [letter for letter in string.ascii_uppercase]+['A'+letter for letter in string.ascii_uppercase]
        alphabet_dict = {char:idx for idx, char in enumerate(spreadsheet_columns)}
        for column_name, column_number in empty_columns.items():
            empty_column_loc = alphabet_dict[empty_columns[column_name].upper()] -1
            new_results.insert(loc=empty_column_loc, column=column_name, value='')
            print(f'{empty_columns[column_name].upper()} ({empty_column_loc}): {column_name}', end=', ')
        new_results.columns = [
            f'{spreadsheet_columns[index+1]}: {column}' for index, column in enumerate(new_results.columns)
            ]

    print(f'\n** Merged dataframe shape:', new_results.shape)
    print([column for column in new_results.columns])
    chain_results_dict[iteration_id] = new_results
    try:
        original_summary_time = next(iter(chatbot_dict[iteration_id].values())).date_created
    except:
        original_summary_time = 'previous_summaries'
    print(f'Original summary time: {original_summary_time}')
    if save_df:
        try:
            save_output(
                chain_results_dict[iteration_id], description=f'batch_Chaining_attributes_{original_summary_time}_append_{results_type}',
                csv_path=csv_path, pickle_path=pickle_path)
            print('')
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} DataFrame')
    if save_chatbot:
        try:
            print('Saving Chaining object (chatbot)...')
            save_instance_to_dict(
                chatbot_dict[iteration_id], 
                description=f'batch_Chaining_attributes_{original_summary_time}_append_{results_type}',
                pickle_path=pickle_path, json_path=json_path
                )
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} chatbot')
            
    return chain_results_dict

chain_results_dict = dict()
iteration_id = 1

chain_results_dict = merge_chaining_results(
    chain_results_dict, chatbot_dict, iteration_id, pivot=False
    )
# chain_results_dict[iteration_id].filter(regex='article|summary|statement|full')


Processing text1_prompt00...
No previous simple prompts for text1_prompt00
No previous relevance prompts for text1_prompt00



Processing text1_prompt01...
No previous simple prompts for text1_prompt01
No previous relevance prompts for text1_prompt01



Processing text1_prompt02...
No previous simple prompts for text1_prompt02
No previous relevance prompts for text1_prompt02



Processing text2_prompt00...
No previous simple prompts for text2_prompt00
No previous relevance prompts for text2_prompt00



Processing text2_prompt01...
No previous simple prompts for text2_prompt01
No previous relevance prompts for text2_prompt01



Processing text2_prompt02...
No previous simple prompts for text2_prompt02
No previous relevance prompts for text2_prompt02



Original summaries DataFrame shape: (30, 9)
Original summaries Dataframe columns: Index(['choice', 'article_title', 'system_role', 'model', 'text', 'prep step',
       'summarization task', 'full summarization task', 'summary'],
      dty

### example dataframe without added empty columns

In [78]:
chain_results_dict[iteration_id]

,choice,article_title,system_role,model,text,prep step,summarization task,full summarization task,summary,audience simplify,full simplify task,simple summary,simple summary choice,simplify task,audience relevance,full relevance task,relevance choice,relevance statement,relevance task
0,0,The Impact of Sleep Duration on Performance Among Competitive Athletes: A Systematic Literature ...,You are an expert at science communication.,gpt-3.5-turbo,The Impact of Sleep Duration on Performance Among Competitive Athletes: A Systematic Literature ...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post.,Take the key points and numerical descriptors to Summarize for a LinkedIn post.,A systematic literature review found no negative association between sleep duration and athletic...,people without a science background,"If needed, rewrite the text using terms appropriate for the audience. If not keep it the same. ...",A recent study found that getting enough sleep doesn't negatively affect how well athletes perfo...,1,"If needed, rewrite the text using terms appropriate for the audience. If not keep it the same. ...",seniors,Rewrite this summary to include a statement of how it is relevant for the audience. Foll...,1,"A recent study found that getting enough sleep is important for athletic performance, especially...",Rewrite this summary to include a statement of how it is relevant for the audience. Foll...
1,0,The Impact of Sleep Duration on Performance Among Competitive Athletes: A Systematic Literature ...,You are an expert at science communication.,gpt-3.5-turbo,The Impact of Sleep Duration on Performance Among Competitive Athletes: A Systematic Literature ...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post.,Take the key points and numerical descriptors to Summarize for a LinkedIn post.,A systematic literature review found no negative association between sleep duration and athletic...,people without a science background,"If needed, rewrite the text using terms appropriate for the audience. If not keep it the same. ...",A recent study found that getting enough sleep doesn't negatively affect how well athletes perfo...,1,"If needed, rewrite the text using terms appropriate for the audience. If not keep it the same. ...",people who enjoy sports,Rewrite this summary to include a statement of how it is relevant for the audience. Foll...,1,"Getting enough sleep is important for athletic performance, especially for aerobic sports. A rev...",Rewrite this summary to include a statement of how it is relevant for the audience. Foll...
2,1,The Impact of Sleep Duration on Performance Among Competitive Athletes: A Systematic Literature ...,You are an expert at science communication.,gpt-3.5-turbo,The Impact of Sleep Duration on Performance Among Competitive Athletes: A Systematic Literature ...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post.,Take the key points and numerical descriptors to Summarize for a LinkedIn post.,"""Sleep duration has a positive impact on athletic performance, according to a systematic literat...",people without a science background,"If needed, rewrite the text using terms appropriate for the audience. If not keep it the same. ...","""Getting enough sleep can help you perform better in sports, according to a review of many studi...",1,"If needed, rewrite the text using terms appropriate for the audience. If not keep it the same. ...",seniors,Rewrite this summary to include a statement of how it is relevant for the audience. Foll...,1,"Sleeping well can help seniors improve their athletic performance, according to a review of publ...",Rewrite this summary to include a statement of how it is relevant for the audience. Foll...
3,1,The Impact of Sleep Duration on Performance Among Competitive Athletes: A Systematic Literature ...,You are an expert at science communication.,gpt-3.5-turbo,The Impact of Sleep Duration on Performance Among Competi

## Iteration 4 Make it pivot

In [81]:
import string
def merge_chaining_results(
        chain_results_dict, chatbot_dict, iteration_id, 
        empty_columns=None, pivot=True, validate=None,
        chatbot_id=None, save_df=False, save_chatbot=False, 
        csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
        pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles',
        json_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\json'
        ):
    """
    Create a dataframe of new 'simple' or 'relevance' summaries from a Chaining object.
    Merge it with the original summaries DataFrame.

    Parameters:
        - chain_results_dict (dict): dictionary of DataFrames.
        - qna_dict (dict): dictionary of QnA DataFrames.
        - chatbot_dict (dict): dictionary of Chaining objects.
        - iteration_id (int, float, or string): iteration_id (dict key) of the chatbot_dict to process.
        - results_type (str): 'simple' or 'relevance'. Default is 'simple'.
        - empty_columns (Bool, int, or dict): dictionary of empty columns to add to the DataFrame. 
            If True or 1, default dictionary is used.
            If False or 0, no empty columns are added.
        - pivot (Bool): whether to pivot the relevance summaries DataFrame. Default is True.
        - validate (str): Argument to pass to pd.merge() to validate the merge.
        - chatbot_id (int, float, or string): chatbot_id (dict key) of the chatbot_dict to process.
        - save_df, save_chatbot (Bool): whether to save the DataFrame and chatbot_dict.

    """
    df_list_simple = []
    df_list_relevance = []
    qna_dfs_list = []
    iteration_id = chatbot_id if chatbot_id != None else iteration_id
    for chatbot_key in chatbot_dict[iteration_id].keys():
        print(f'Processing {chatbot_key}...')
        try: 
            n_previous_prompts_simple = chatbot_dict[iteration_id][chatbot_key].previous_n_prompts['simple']
            print(f'\tNumber of previous relevance prompts: {n_previous_prompts_simple}')
        except:
            n_previous_prompts_simple = 0
            print(f'No previous simple prompts for {chatbot_key}')
        try: 
            n_previous_prompts_relevance = chatbot_dict[iteration_id][chatbot_key].previous_n_prompts['relevance']
            print(f'\tNumber of previous relevance prompts: {n_previous_prompts_relevance}')
        except:
            n_previous_prompts_relevance = 0
            print(f'No previous relevance prompts for {chatbot_key}')
            
        # qna_dfs_list.append(pd.DataFrame(chatbot_dict[iteration_id][chatbot_key].qna))
        # qna_dfs_list.append(pd.DataFrame(chatbot_dict[iteration_id][chatbot_key].qna).reset_index(inplace=True, names=['choice']))
        qna_dfs_list.append(pd.DataFrame(chatbot_dict[iteration_id][chatbot_key].qna).reset_index(names=['choice']))

        # create results dictionaries that only grabs the results of the new prompts instead of all
        results_dict_simple = dict()
        total_n_prompts_simple = len(chatbot_dict[iteration_id][chatbot_key].simple_summary_dict)
        for prompt_number in range(n_previous_prompts_simple+1, total_n_prompts_simple+1):
            results_dict_simple[prompt_number] = chatbot_dict[iteration_id][chatbot_key].simple_summary_dict[prompt_number]
        chatbot_dict[iteration_id][chatbot_key].simple_summary_dict

        results_dict_relevance = dict()
        total_n_prompts_relevance = len(chatbot_dict[iteration_id][chatbot_key].relevance_dict)
        for prompt_number_relevance in range(n_previous_prompts_relevance+1, total_n_prompts_relevance+1):
            # print(f'\tAppending results for prompt {prompt_number_relevance} of {total_n_prompts_relevance}')
            results_dict_relevance[prompt_number_relevance] = chatbot_dict[iteration_id][chatbot_key].relevance_dict[prompt_number_relevance]

        for iteration_key_simple in results_dict_simple.keys():
            response_keys_simple = sorted([text_prompt_key for text_prompt_key in results_dict_simple[iteration_key_simple].keys()])
            # print(f'\tAppending results for {iteration_key_simple}: ', end='')

            for response_key_simple in response_keys_simple:
                df_list_simple.append(pd.DataFrame(results_dict_simple[iteration_key_simple][response_key_simple]).transpose())
                # print(f'{response_key_simple}, ', end='')
            # print('')
        for iteration_key_relevance in results_dict_relevance.keys():
            response_keys_relevance = sorted([text_prompt_key for text_prompt_key in results_dict_relevance[iteration_key_relevance].keys()])
            for response_key_relevance in response_keys_relevance:
                df_list_relevance.append(pd.DataFrame(results_dict_relevance[iteration_key_relevance][response_key_relevance]).transpose())
                # print(f'{response_key_relevance}, ', end='')
            # print('')

    
    simple_summary_df = pd.concat(df_list_simple)
    relevance_df = pd.concat(df_list_relevance)
    qna_df = pd.concat(qna_dfs_list)
    print(f'Original summaries DataFrame shape: {qna_df.shape}')
    print(f'Original summaries Dataframe columns: {qna_df.columns}')
    print('Simple summaries DataFrame shape:', simple_summary_df.shape)
    print(f'\tSimple summaries DataFrame columns: {[col for col in simple_summary_df.columns]}')
    print('Relevance summaries DataFrame shape:', relevance_df.shape)
    print(f'\tRelevance summaries DataFrame columns: {[col for col in relevance_df.columns]}')
    print(f'Unique relevance audience values: {relevance_df.audience.unique()}')
    
    original_summary_columns = [
        'article_title',
        # 'choice',
        'system_role',
        'model',
        'text',
        'prep step',
        'summarization task',
        'full summarization task',
        'summary',
    ]
    
    common_columns = list(set(simple_summary_df.columns) & set(relevance_df.columns))
    common_columns.remove('audience')
    new_results = qna_df.merge(
        simple_summary_df, how='right',
        right_on='original summary',
        left_on='summary',
        validate=validate
        ).drop(columns='original summary')
    if pivot == False:
        validate=None
        
        print(f'DataFrame shape after merging with simple summaries: {new_results.shape}')
        new_results= new_results.merge(
            relevance_df, how='outer', suffixes=(' simplify', ' relevance'),
            left_on='summary', right_on='preceding summary', validate=validate
            ).drop(columns='preceding summary')
    else:
        relevance_pivot_df = relevance_df.pivot(
            columns=['audience'],
            values='relevance statement',
            index=['preceding summary', 'relevance task']
        ).reset_index()
        new_results = new_results.merge(
            relevance_pivot_df, how='outer', suffixes=(' simplify', ' relevance'),
            left_on='summary', right_on='preceding summary',
            validate='m:1' if validate else None
        )

    if empty_columns:
        if pivot == False:
            if (type(empty_columns) != dict):
                empty_columns = {
                    "choice numnber": "C",
                    "original summary content rating": "K",
                    "original summary language rating": "L",
                    "top summary": "M",
                    "simplify audience": "O",
                    "simplify task": "P",
                    "full simplify task": "Q",
                    "simple summary": "R",
                    "simple summary content rating": "S",
                    "simple summary language rating": "T",
                    "top simple summary": "U",
                }
        else:           
            if (type(empty_columns) != dict):
                empty_columns = {
                    "choice numnber": "C",
                        "original summary content rating": "K",
                    "original summary language rating": "L",
                    "top summary": "M",
                    "simple summary choice": "N",
                    "simplify audience": "O",
                    "simplify task": "P",
                    "full simplify task": "Q",
                    "simple summary": "R",
                    "simple summary content rating": "S",
                    "simple summary language rating": "T",
                    'top simple summary': 'u',
                    'full add relevance task': 'w',
                    'added relevance content rating': 'y',
                    'added relevance language rating': 'z',
                    'top added relevance': 'aa',
                    'add relevance task (seniors)': 'AB',
                    'full add relevance task (seniors)': 'AC',
                }
        print(f'Merged DataFrame shape: {new_results.shape}')
        print('\nColumns before adding empty columns:', [column for column in new_results.columns])
        print('Inserting empty columns...', end='\n\t')
        spreadsheet_columns = [letter for letter in string.ascii_uppercase]+['A'+letter for letter in string.ascii_uppercase]
        alphabet_dict = {char:idx for idx, char in enumerate(spreadsheet_columns)}
        for column_name, column_number in empty_columns.items():
            empty_column_loc = alphabet_dict[empty_columns[column_name].upper()] -1
            new_results.insert(loc=empty_column_loc, column=column_name, value='')
            print(f'{empty_columns[column_name].upper()} ({empty_column_loc}): {column_name}', end=', ')
        new_results.columns = [
            f'{spreadsheet_columns[index+1]}: {column}' for index, column in enumerate(new_results.columns)
            ]

    print(f'\n** Merged dataframe shape:', new_results.shape)
    print([column for column in new_results.columns])
    chain_results_dict[iteration_id] = new_results
    try:
        original_summary_time = next(iter(chatbot_dict[iteration_id].values())).date_created
    except:
        original_summary_time = 'previous_summaries'
    print(f'Original summary time: {original_summary_time}')
    if save_df:
        try:
            save_output(
                chain_results_dict[iteration_id], description=f'batch_Chaining_attributes_{original_summary_time}_append_{results_type}',
                csv_path=csv_path, pickle_path=pickle_path)
            print('')
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} DataFrame')
    if save_chatbot:
        try:
            print('Saving Chaining object (chatbot)...')
            save_instance_to_dict(
                chatbot_dict[iteration_id], 
                description=f'batch_Chaining_attributes_{original_summary_time}_append_{results_type}',
                pickle_path=pickle_path, json_path=json_path
                )
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} chatbot')
            
    return chain_results_dict

chain_results_dict = dict()
iteration_id = 1

chain_results_dict = merge_chaining_results(
    chain_results_dict, chatbot_dict, iteration_id, pivot=True
    )
# chain_results_dict[iteration_id].filter(regex='article|summary|statement|full')


Processing text1_prompt00...
No previous simple prompts for text1_prompt00
No previous relevance prompts for text1_prompt00



Processing text1_prompt01...
No previous simple prompts for text1_prompt01
No previous relevance prompts for text1_prompt01



Processing text1_prompt02...
No previous simple prompts for text1_prompt02
No previous relevance prompts for text1_prompt02



Processing text2_prompt00...
No previous simple prompts for text2_prompt00
No previous relevance prompts for text2_prompt00



Processing text2_prompt01...
No previous simple prompts for text2_prompt01
No previous relevance prompts for text2_prompt01



Processing text2_prompt02...
No previous simple prompts for text2_prompt02
No previous relevance prompts for text2_prompt02



Original summaries DataFrame shape: (30, 9)
Original summaries Dataframe columns: Index(['choice', 'article_title', 'system_role', 'model', 'text', 'prep step',
       'summarization task', 'full summarization task', 'summary'],
      dty

In [82]:
chain_results_dict[iteration_id]

,choice,article_title,system_role,model,text,prep step,summarization task,full summarization task,summary,audience,full simplify task,simple summary,simple summary choice,simplify task,preceding summary,relevance task,people who enjoy sports,seniors
0,0,The Impact of Sleep Duration on Performance Among Competitive Athletes: A Systematic Literature ...,You are an expert at science communication.,gpt-3.5-turbo,The Impact of Sleep Duration on Performance Among Competitive Athletes: A Systematic Literature ...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post.,Take the key points and numerical descriptors to Summarize for a LinkedIn post.,A systematic literature review found no negative association between sleep duration and athletic...,people without a science background,"If needed, rewrite the text using terms appropriate for the audience. If not keep it the same. ...",A recent study found that getting enough sleep doesn't negatively affect how well athletes perfo...,1,"If needed, rewrite the text using terms appropriate for the audience. If not keep it the same. ...",A systematic literature review found no negative association between sleep duration and athletic...,Rewrite this summary to include a statement of how it is relevant for the audience. Foll...,"Getting enough sleep is important for athletic performance, especially for aerobic sports. A rev...","A recent study found that getting enough sleep is important for athletic performance, especially..."
1,1,The Impact of Sleep Duration on Performance Among Competitive Athletes: A Systematic Literature ...,You are an expert at science communication.,gpt-3.5-turbo,The Impact of Sleep Duration on Performance Among Competitive Athletes: A Systematic Literature ...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post.,Take the key points and numerical descriptors to Summarize for a LinkedIn post.,"""Sleep duration has a positive impact on athletic performance, according to a systematic literat...",people without a science background,"If needed, rewrite the text using terms appropriate for the audience. If not keep it the same. ...","""Getting enough sleep can help you perform better in sports, according to a review of many studi...",1,"If needed, rewrite the text using terms appropriate for the audience. If not keep it the same. ...","""Sleep duration has a positive impact on athletic performance, according to a systematic literat...",Rewrite this summary to include a statement of how it is relevant for the audience. Foll...,"Getting enough sleep can positively impact athletic performance, according to a review of publis...","Sleeping well can help seniors improve their athletic performance, according to a review of publ..."
2,2,The Impact of Sleep Duration on Performance Among Competitive Athletes: A Systematic Literature ...,You are an expert at science communication.,gpt-3.5-turbo,The Impact of Sleep Duration on Performance Among Competitive Athletes: A Systematic Literature ...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post.,Take the key points and numerical descriptors to Summarize for a LinkedIn post.,"""New literature review shows that increased sleep duration has a positive effect on athletic per...",people without a science background,"If needed, rewrite the text using terms appropriate for the audience. If not keep it the same. ...",A new study shows that getting more sleep can help athletes perform better. The study found that...,1,"If needed, rewrite the text using terms appropriate for the audience. If not keep it the same. ...","""New literature review shows that increased sleep duration has a positive effect on athletic per...",Rewrite this summary to include a statement of how it is relevant for the audience. Foll...,Getting enough sleep is crucial for athletes to perform at their best. A recent literature revie...,A new literature review suggests that getting more sleep can have

## Iteration 5 empty columns

In [99]:
import string
def merge_chaining_results(
        chain_results_dict, chatbot_dict, iteration_id, 
        empty_columns=None, pivot=True, validate=None,
        chatbot_id=None, save_df=False, save_chatbot=False, 
        csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
        pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles',
        json_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\json'
        ):
    """
    Create a dataframe of new 'simple' or 'relevance' summaries from a Chaining object.
    Merge it with the original summaries DataFrame.

    Parameters:
        - chain_results_dict (dict): dictionary of DataFrames.
        - qna_dict (dict): dictionary of QnA DataFrames.
        - chatbot_dict (dict): dictionary of Chaining objects.
        - iteration_id (int, float, or string): iteration_id (dict key) of the chatbot_dict to process.
        - results_type (str): 'simple' or 'relevance'. Default is 'simple'.
        - empty_columns (Bool, int, or dict): dictionary of empty columns to add to the DataFrame. 
            If True or 1, default dictionary is used.
            If False or 0, no empty columns are added.
        - pivot (Bool): whether to pivot the relevance summaries DataFrame. Default is True.
        - validate (str): Argument to pass to pd.merge() to validate the merge.
        - chatbot_id (int, float, or string): chatbot_id (dict key) of the chatbot_dict to process.
        - save_df, save_chatbot (Bool): whether to save the DataFrame and chatbot_dict.

    """
    df_list_simple = []
    df_list_relevance = []
    qna_dfs_list = []
    iteration_id = chatbot_id if chatbot_id != None else iteration_id
    for chatbot_key in chatbot_dict[iteration_id].keys():
        print(f'Processing {chatbot_key}...')
        try: 
            n_previous_prompts_simple = chatbot_dict[iteration_id][chatbot_key].previous_n_prompts['simple']
            print(f'\tNumber of previous relevance prompts: {n_previous_prompts_simple}')
        except:
            n_previous_prompts_simple = 0
            print(f'No previous simple prompts for {chatbot_key}')
        try: 
            n_previous_prompts_relevance = chatbot_dict[iteration_id][chatbot_key].previous_n_prompts['relevance']
            print(f'\tNumber of previous relevance prompts: {n_previous_prompts_relevance}')
        except:
            n_previous_prompts_relevance = 0
            print(f'No previous relevance prompts for {chatbot_key}')
            
        # qna_dfs_list.append(pd.DataFrame(chatbot_dict[iteration_id][chatbot_key].qna))
        # qna_dfs_list.append(pd.DataFrame(chatbot_dict[iteration_id][chatbot_key].qna).reset_index(inplace=True, names=['choice']))
        qna_dfs_list.append(pd.DataFrame(chatbot_dict[iteration_id][chatbot_key].qna).reset_index(names=['choice']))

        # create results dictionaries that only grabs the results of the new prompts instead of all
        results_dict_simple = dict()
        total_n_prompts_simple = len(chatbot_dict[iteration_id][chatbot_key].simple_summary_dict)
        for prompt_number in range(n_previous_prompts_simple+1, total_n_prompts_simple+1):
            results_dict_simple[prompt_number] = chatbot_dict[iteration_id][chatbot_key].simple_summary_dict[prompt_number]
        chatbot_dict[iteration_id][chatbot_key].simple_summary_dict

        results_dict_relevance = dict()
        total_n_prompts_relevance = len(chatbot_dict[iteration_id][chatbot_key].relevance_dict)
        for prompt_number_relevance in range(n_previous_prompts_relevance+1, total_n_prompts_relevance+1):
            # print(f'\tAppending results for prompt {prompt_number_relevance} of {total_n_prompts_relevance}')
            results_dict_relevance[prompt_number_relevance] = chatbot_dict[iteration_id][chatbot_key].relevance_dict[prompt_number_relevance]

        for iteration_key_simple in results_dict_simple.keys():
            response_keys_simple = sorted([text_prompt_key for text_prompt_key in results_dict_simple[iteration_key_simple].keys()])
            # print(f'\tAppending results for {iteration_key_simple}: ', end='')

            for response_key_simple in response_keys_simple:
                df_list_simple.append(pd.DataFrame(results_dict_simple[iteration_key_simple][response_key_simple]).transpose())
                # print(f'{response_key_simple}, ', end='')
            # print('')
        for iteration_key_relevance in results_dict_relevance.keys():
            response_keys_relevance = sorted([text_prompt_key for text_prompt_key in results_dict_relevance[iteration_key_relevance].keys()])
            for response_key_relevance in response_keys_relevance:
                df_list_relevance.append(pd.DataFrame(results_dict_relevance[iteration_key_relevance][response_key_relevance]).transpose())
                # print(f'{response_key_relevance}, ', end='')
            # print('')

    
    simple_summary_df = pd.concat(df_list_simple)
    relevance_df = pd.concat(df_list_relevance)
    qna_df = pd.concat(qna_dfs_list)
    print(f'Original summaries DataFrame shape: {qna_df.shape}')
    print(f'Original summaries Dataframe columns: {qna_df.columns}')
    print('Simple summaries DataFrame shape:', simple_summary_df.shape)
    print(f'\tSimple summaries DataFrame columns: {[col for col in simple_summary_df.columns]}')
    print('Relevance summaries DataFrame shape:', relevance_df.shape)
    print(f'\tRelevance summaries DataFrame columns: {[col for col in relevance_df.columns]}')
    print(f'Unique relevance audience values: {relevance_df.audience.unique()}')
    
    
    common_columns = list(set(simple_summary_df.columns) & set(relevance_df.columns))
    common_columns.remove('audience')
    new_results = qna_df.merge(
        simple_summary_df, how='right',
        right_on='original summary',
        left_on='summary',
        validate=validate
        ).drop(columns='original summary')
    if pivot == False:
        validate=None
        
        print(f'DataFrame shape after merging with simple summaries: {new_results.shape}')
        new_results= new_results.merge(
            relevance_df, how='outer', suffixes=(' simplify', ' relevance'),
            left_on='summary', right_on='preceding summary', validate=validate
            ).drop(columns='preceding summary')
    else:
        relevance_pivot_df = relevance_df.pivot(
            columns=['audience'],
            values='relevance statement',
            index=['preceding summary', 'relevance task']
        ).reset_index()
        new_results = new_results.merge(
            relevance_pivot_df, how='outer', suffixes=(' simplify', ' relevance'),
            left_on='summary', right_on='preceding summary',
            validate='m:1' if validate else None
        ).drop(columns='preceding summary')

    spreadsheet_columns = [
        "choice",
        "article_title",
        "system_role",
        "model",
        "text",
        "prep step",
        "summarization task",
        "full summarization task",
        "summary",
        "simple summary choice",
        "audience",
        "simplify task",
        "full simplify task",
        "simple summary",
        "relevance task",
        "people who enjoy sports",
        "seniors",
    ]
    new_results = new_results[spreadsheet_columns]
    if empty_columns:
        if pivot == False:
            if (type(empty_columns) != dict):
                empty_columns = {
                    "choice numnber": "C",
                    "original summary content rating": "K",
                    "original summary language rating": "L",
                    "top summary": "M",
                    "simplify audience": "O",
                    # "simplify task": "P",
                    "full simplify task": "Q",
                    "simple summary": "R",
                    "simple summary content rating": "S",
                    "simple summary language rating": "T",
                    "top simple summary": "U",
                }
        else:           
            if (type(empty_columns) != dict):
                empty_columns = {
                    # "choice numnber": "C",
                    "original summary content rating": "K",
                    "original summary language rating": "L",
                    "top summary": "M",
                    # "simple summary choice": "N",
                    # "simplify audience": "O",
                    # "simplify task": "P",
                    # "full simplify task": "Q",
                    # "simple summary": "R",
                    "simple summary content rating": "S",
                    "simple summary language rating": "T",
                    'top simple summary': 'u',
                    # 'full add relevance task': 'w',
                    'added relevance content rating': 'y',
                    'added relevance language rating': 'z',
                    'top added relevance': 'aa',
                    'add relevance task (seniors)': 'AB',
                    'full add relevance task (seniors)': 'AC',
                }
        print(f'Merged DataFrame shape: {new_results.shape}')
        print('\nColumns before adding empty columns:', [column for column in new_results.columns])
        print('Inserting empty columns...', end='\n\t')
        spreadsheet_columns = [letter for letter in string.ascii_uppercase]+['A'+letter for letter in string.ascii_uppercase]
        alphabet_dict = {char:idx for idx, char in enumerate(spreadsheet_columns)}
        for column_name, column_number in empty_columns.items():
            empty_column_loc = alphabet_dict[empty_columns[column_name].upper()] -1
            new_results.insert(loc=empty_column_loc, column=column_name, value='')
            print(f'{empty_columns[column_name].upper()} ({empty_column_loc}): {column_name}', end=', ')
        new_results.columns = [
            f'{spreadsheet_columns[index+1]}: {column}' for index, column in enumerate(new_results.columns)
            ]

    print(f'\n** Merged dataframe shape:', new_results.shape)
    print([column for column in new_results.columns])
    chain_results_dict[iteration_id] = new_results
    try:
        original_summary_time = next(iter(chatbot_dict[iteration_id].values())).date_created
    except:
        original_summary_time = 'previous_summaries'
    print(f'Original summary time: {original_summary_time}')
    if save_df:
        try:
            save_output(
                chain_results_dict[iteration_id], description=f'batch_Chaining_attributes_{original_summary_time}_append_{results_type}',
                csv_path=csv_path, pickle_path=pickle_path)
            print('')
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} DataFrame')
    if save_chatbot:
        try:
            print('Saving Chaining object (chatbot)...')
            save_instance_to_dict(
                chatbot_dict[iteration_id], 
                description=f'batch_Chaining_attributes_{original_summary_time}_append_{results_type}',
                pickle_path=pickle_path, json_path=json_path
                )
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} chatbot')
            
    return chain_results_dict

chain_results_dict = dict()
iteration_id = 1

chain_results_dict = merge_chaining_results(
    chain_results_dict, chatbot_dict, iteration_id, pivot=True,
    empty_columns=True
    )
chain_results_dict[iteration_id].head()
# chain_results_dict[iteration_id].filter(regex='article|summary|statement|full')


Processing text1_prompt00...
No previous simple prompts for text1_prompt00
No previous relevance prompts for text1_prompt00
Processing text1_prompt01...
No previous simple prompts for text1_prompt01
No previous relevance prompts for text1_prompt01
Processing text1_prompt02...
No previous simple prompts for text1_prompt02
No previous relevance prompts for text1_prompt02
Processing text2_prompt00...
No previous simple prompts for text2_prompt00
No previous relevance prompts for text2_prompt00
Processing text2_prompt01...
No previous simple prompts for text2_prompt01
No previous relevance prompts for text2_prompt01
Processing text2_prompt02...
No previous simple prompts for text2_prompt02
No previous relevance prompts for text2_prompt02
Original summaries DataFrame shape: (30, 9)
Original summaries Dataframe columns: Index(['choice', 'article_title', 'system_role', 'model', 'text', 'prep step',
       'summarization task', 'full summarization task', 'summary'],
      dtype='object')
Simpl

,B: choice,C: article_title,D: system_role,E: model,F: text,G: prep step,H: summarization task,I: full summarization task,J: summary,K: original summary content rating,L: original summary language rating,M: top summary,N: simple summary choice,O: audience,P: simplify task,Q: full simplify task,R: simple summary,S: simple summary content rating,T: simple summary language rating,U: top simple summary,V: relevance task,W: people who enjoy sports,X: seniors,Y: added relevance content rating,Z: added relevance language rating,AA: top added relevance,AB: add relevance task (seniors),AC: full add relevance task (seniors)
0,0,The Impact of Sleep Duration on Performance Among Competitive Athletes: A Systematic Literature ...,You are an expert at science communication.,gpt-3.5-turbo,The Impact of Sleep Duration on Performance Among Competitive Athletes: A Systematic Literature ...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post.,Take the key points and numerical descriptors to Summarize for a LinkedIn post.,A systematic literature review found no negative association between sleep duration and athletic...,,,,1,people without a science background,"If needed, rewrite the text using terms appropriate for the audience. If not keep it the same. ...","If needed, rewrite the text using terms appropriate for the audience. If not keep it the same. ...",A recent study found that getting enough sleep doesn't negatively affect how well athletes perfo...,,,,Rewrite this summary to include a statement of how it is relevant for the audience. Foll...,"Getting enough sleep is important for athletic performance, especially for aerobic sports. A rev...","A recent study found that getting enough sleep is important for athletic performance, especially...",,,,,
1,1,The Impact of Sleep Duration on Performance Among Competitive Athletes: A Systematic Literature ...,You are an expert at science communication.,gpt-3.5-turbo,The Impact of Sleep Duration on Performance Among Competitive Athletes: A Systematic Literature ...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post.,Take the key points and numerical descriptors to Summarize for a LinkedIn post.,"""Sleep duration has a positive impact on athletic performance, according to a systematic literat...",,,,1,people without a science background,"If needed, rewrite the text using terms appropriate for the audience. If not keep it the same. ...","If needed, rewrite the text using terms appropriate for the audience. If not keep it the same. ...","""Getting enough sleep can help you perform better in sports, according to a review of many studi...",,,,Rewrite this summary to include a statement of how it is relevant for the audience. Foll...,"Getting enough sleep can positively impact athletic performance, according to a review of publis...","Sleeping well can help seniors improve their athletic performance, according to a review of publ...",,,,,
2,2,The Impact of Sleep Duration on Performance Among Competitive Athletes: A Systematic Literature ...,You are an expert at science communication.,gpt-3.5-turbo,The Impact of Sleep Duration on Performance Among Competitive Athletes: A Systematic Literature ...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post.,Take the key points and numerical descriptors to Summarize for a LinkedIn post.,"""New literature review shows that increased sleep duration has a positive effect on athletic per...",,,,1,people without a science background,"If needed, rewrite the text using terms appropriate for the audience. If not keep it the same. ...","If needed, rewrite the text using terms appropriate for the audience. If not keep it the same. ...",A new study shows that getting more sleep can help athletes perform better. The study found that...,,,,Rewrite this summary to include a statement of how it is relevant for the audience. Foll...,Getting enough sleep is crucial for athletes to perform at their best. 

In [95]:
for column in chain_results_dict[iteration_id].columns:
    print(f'"{column}",')

"choice",
"article_title",
"system_role",
"model",
"text",
"prep step",
"summarization task",
"full summarization task",
"summary",
"audience",
"full simplify task",
"simple summary",
"simple summary choice",
"simplify task",
"relevance task",
"people who enjoy sports",
"seniors",


In [91]:
chain_results_dict[iteration_id]

,B: choice,C: article_title,D: system_role,E: model,F: text,G: prep step,H: summarization task,I: full summarization task,J: summary,K: original summary content rating,L: original summary language rating,M: top summary,N: audience,O: simplify audience,P: full simplify task,Q: simple summary,R: simple summary choice,S: simple summary content rating,T: simple summary language rating,U: top simple summary,V: simplify task,W: relevance task,X: people who enjoy sports,Y: added relevance content rating,Z: added relevance language rating,AA: top added relevance,AB: add relevance task (seniors),AC: full add relevance task (seniors),AD: seniors
0,0,The Impact of Sleep Duration on Performance Among Competitive Athletes: A Systematic Literature ...,You are an expert at science communication.,gpt-3.5-turbo,The Impact of Sleep Duration on Performance Among Competitive Athletes: A Systematic Literature ...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post.,Take the key points and numerical descriptors to Summarize for a LinkedIn post.,A systematic literature review found no negative association between sleep duration and athletic...,,,,people without a science background,,"If needed, rewrite the text using terms appropriate for the audience. If not keep it the same. ...",A recent study found that getting enough sleep doesn't negatively affect how well athletes perfo...,1,,,,"If needed, rewrite the text using terms appropriate for the audience. If not keep it the same. ...",Rewrite this summary to include a statement of how it is relevant for the audience. Foll...,"Getting enough sleep is important for athletic performance, especially for aerobic sports. A rev...",,,,,,"A recent study found that getting enough sleep is important for athletic performance, especially..."
1,1,The Impact of Sleep Duration on Performance Among Competitive Athletes: A Systematic Literature ...,You are an expert at science communication.,gpt-3.5-turbo,The Impact of Sleep Duration on Performance Among Competitive Athletes: A Systematic Literature ...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post.,Take the key points and numerical descriptors to Summarize for a LinkedIn post.,"""Sleep duration has a positive impact on athletic performance, according to a systematic literat...",,,,people without a science background,,"If needed, rewrite the text using terms appropriate for the audience. If not keep it the same. ...","""Getting enough sleep can help you perform better in sports, according to a review of many studi...",1,,,,"If needed, rewrite the text using terms appropriate for the audience. If not keep it the same. ...",Rewrite this summary to include a statement of how it is relevant for the audience. Foll...,"Getting enough sleep can positively impact athletic performance, according to a review of publis...",,,,,,"Sleeping well can help seniors improve their athletic performance, according to a review of publ..."
2,2,The Impact of Sleep Duration on Performance Among Competitive Athletes: A Systematic Literature ...,You are an expert at science communication.,gpt-3.5-turbo,The Impact of Sleep Duration on Performance Among Competitive Athletes: A Systematic Literature ...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post.,Take the key points and numerical descriptors to Summarize for a LinkedIn post.,"""New literature review shows that increased sleep duration has a positive effect on athletic per...",,,,people without a science background,,"If needed, rewrite the text using terms appropriate for the audience. If not keep it the same. ...",A new study shows that getting more sleep can help athletes perform better. The study found that...,1,,,,"If needed, rewrite the text using terms appropriate for the audience. If not keep it the same. ...",Rewrite this summary to include a statement of how it is relevant for the audience. Foll...,Getting enough sleep is crucial for athletes t

# Iteration 6 add full relevance task and relevance audience columns

In [145]:
import string
def merge_chaining_results(
        chain_results_dict, chatbot_dict, iteration_id, 
        empty_columns=None, pivot=True, validate=None,
        chatbot_id=None, save_df=False, save_chatbot=False, 
        csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
        pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles',
        json_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\json'
        ):
    """
    Create a dataframe of new 'simple' or 'relevance' summaries from a Chaining object.
    Merge it with the original summaries DataFrame.

    Parameters:
        - chain_results_dict (dict): dictionary of DataFrames.
        - qna_dict (dict): dictionary of QnA DataFrames.
        - chatbot_dict (dict): dictionary of Chaining objects.
        - iteration_id (int, float, or string): iteration_id (dict key) of the chatbot_dict to process.
        - results_type (str): 'simple' or 'relevance'. Default is 'simple'.
        - empty_columns (Bool, int, or dict): dictionary of empty columns to add to the DataFrame. 
            If True or 1, default dictionary is used.
            If False or 0, no empty columns are added.
        - pivot (Bool): whether to pivot the relevance summaries DataFrame. Default is True.
        - validate (str): Argument to pass to pd.merge() to validate the merge.
        - chatbot_id (int, float, or string): chatbot_id (dict key) of the chatbot_dict to process.
        - save_df, save_chatbot (Bool): whether to save the DataFrame and chatbot_dict.

    """
    df_list_simple = []
    df_list_relevance = []
    qna_dfs_list = []
    iteration_id = chatbot_id if chatbot_id != None else iteration_id
    for chatbot_key in chatbot_dict[iteration_id].keys():
        print(f'Processing {chatbot_key}...')
        try: 
            n_previous_prompts_simple = chatbot_dict[iteration_id][chatbot_key].previous_n_prompts['simple']
            print(f'\tNumber of previous relevance prompts: {n_previous_prompts_simple}')
        except:
            n_previous_prompts_simple = 0
            print(f'No previous simple prompts for {chatbot_key}')
        try: 
            n_previous_prompts_relevance = chatbot_dict[iteration_id][chatbot_key].previous_n_prompts['relevance']
            print(f'\tNumber of previous relevance prompts: {n_previous_prompts_relevance}')
        except:
            n_previous_prompts_relevance = 0
            print(f'No previous relevance prompts for {chatbot_key}')
            
        # qna_dfs_list.append(pd.DataFrame(chatbot_dict[iteration_id][chatbot_key].qna))
        # qna_dfs_list.append(pd.DataFrame(chatbot_dict[iteration_id][chatbot_key].qna).reset_index(inplace=True, names=['choice']))
        qna_dfs_list.append(pd.DataFrame(chatbot_dict[iteration_id][chatbot_key].qna).reset_index(names=['choice']))

        # create results dictionaries that only grabs the results of the new prompts instead of all
        results_dict_simple = dict()
        total_n_prompts_simple = len(chatbot_dict[iteration_id][chatbot_key].simple_summary_dict)
        for prompt_number in range(n_previous_prompts_simple+1, total_n_prompts_simple+1):
            results_dict_simple[prompt_number] = chatbot_dict[iteration_id][chatbot_key].simple_summary_dict[prompt_number]
        chatbot_dict[iteration_id][chatbot_key].simple_summary_dict

        results_dict_relevance = dict()
        total_n_prompts_relevance = len(chatbot_dict[iteration_id][chatbot_key].relevance_dict)
        for prompt_number_relevance in range(n_previous_prompts_relevance+1, total_n_prompts_relevance+1):
            # print(f'\tAppending results for prompt {prompt_number_relevance} of {total_n_prompts_relevance}')
            results_dict_relevance[prompt_number_relevance] = chatbot_dict[iteration_id][chatbot_key].relevance_dict[prompt_number_relevance]

        for iteration_key_simple in results_dict_simple.keys():
            response_keys_simple = sorted([text_prompt_key for text_prompt_key in results_dict_simple[iteration_key_simple].keys()])
            # print(f'\tAppending results for {iteration_key_simple}: ', end='')

            for response_key_simple in response_keys_simple:
                df_list_simple.append(pd.DataFrame(results_dict_simple[iteration_key_simple][response_key_simple]).transpose())
                # print(f'{response_key_simple}, ', end='')
            # print('')
        for iteration_key_relevance in results_dict_relevance.keys():
            response_keys_relevance = sorted([text_prompt_key for text_prompt_key in results_dict_relevance[iteration_key_relevance].keys()])
            for response_key_relevance in response_keys_relevance:
                df_list_relevance.append(pd.DataFrame(results_dict_relevance[iteration_key_relevance][response_key_relevance]).transpose())
                # print(f'{response_key_relevance}, ', end='')
            # print('')

    
    simple_summary_df = pd.concat(df_list_simple)
    relevance_df = pd.concat(df_list_relevance)
    qna_df = pd.concat(qna_dfs_list)
    print(f'Original summaries DataFrame shape: {qna_df.shape}')
    print(f'Original summaries Dataframe columns: {qna_df.columns}')
    print('Simple summaries DataFrame shape:', simple_summary_df.shape)
    print(f'\tSimple summaries DataFrame columns: {[col for col in simple_summary_df.columns]}')
    print('Relevance summaries DataFrame shape:', relevance_df.shape)
    print(f'\tRelevance summaries DataFrame columns: {[col for col in relevance_df.columns]}')

    relevance_audience_list = sorted(relevance_df.audience.unique().tolist())
    print(f'Unique relevance audience values: {relevance_audience_list}')
    
    
    common_columns = list(set(simple_summary_df.columns) & set(relevance_df.columns))
    common_columns.remove('audience')
    spreadsheet_columns = [
        "choice",
        "article_title",
        "system_role",
        "model",
        "text",
        "prep step",
        "summarization task",
        "full summarization task",
        "summary",
        "simple summary choice",
        "audience",
        "simplify task",
        "full simplify task",
        "simple summary",
        "relevance task",
        "full relevance task",
        "people who enjoy sports",
        "seniors",
    ]
    new_results = qna_df.merge(
        simple_summary_df, how='right',
        right_on='original summary',
        left_on='summary',
        validate=validate
        ).drop(columns='original summary')
    if pivot == False:
        validate=None
        
        print(f'DataFrame shape after merging with simple summaries: {new_results.shape}')
        new_results= new_results.merge(
            relevance_df, how='outer', suffixes=(' simplify', ' relevance'),
            left_on='summary', right_on='preceding summary', validate=validate
            ).drop(columns='preceding summary')
    else:
        relevance_pivot_df = relevance_df.pivot(
            columns=['audience'],
            values='relevance statement',
            index=['preceding summary', 'relevance task',]
        ).sort_index().reset_index()
        new_results = new_results.merge(
            relevance_pivot_df, how='outer', suffixes=(' simplify', ' relevance'),
            left_on='summary', right_on='preceding summary',
            validate='m:1' if validate else None
        ).drop(columns='preceding summary')
        new_results['full relevance task'] = new_results['relevance task'].apply(lambda x: f'{x} {relevance_audience_list[0]}')
        new_results['add relevance task (seniors)'] = new_results["relevance task"]
        new_results['full add relevance task (seniors)'] =new_results['relevance task'].apply(lambda x: f'{x} {relevance_audience_list[1]}')
        # new_results['full relevance task'] = f"{new_results['relevance task']} {relevance_audience_list[0]}"
        # new_results['add relevance task (seniors)'] = f'{new_results["relevance task"]}'
        # new_results['full add relevance task (seniors)'] = f"{new_results['relevance task']} {relevance_audience_list[1]}"
        spreadsheet_columns.append('add relevance task (seniors)')
        spreadsheet_columns.append('full add relevance task (seniors)')
        spreadsheet_columns.remove(relevance_audience_list[1])
        spreadsheet_columns.append(relevance_audience_list[1])
        
    new_results = new_results[spreadsheet_columns]
    if empty_columns:
        if pivot == False:
            if (type(empty_columns) != dict):
                empty_columns = {
                    # "choice numnber": "C",
                    "original summary content rating": "K",
                    "original summary language rating": "L",
                    "top summary": "M",
                    # "simplify audience": "O",
                    # "simplify task": "P",
                    # "full simplify task": "Q",
                    # "simple summary": "R",
                    "simple summary content rating": "S",
                    "simple summary language rating": "T",
                    "top simple summary": "U",
                }
        else:           
            if (type(empty_columns) != dict):
                empty_columns = {
                    # "choice numnber": "C",
                    "original summary content rating": "K",
                    "original summary language rating": "L",
                    "top summary": "M",
                    # "simple summary choice": "N",
                    # "simplify audience": "O",
                    # "simplify task": "P",
                    # "full simplify task": "Q",
                    # "simple summary": "R",
                    "simple summary content rating": "S",
                    "simple summary language rating": "T",
                    'top simple summary': 'u',
                    # 'full add relevance task': 'w',
                    'added relevance content rating': 'y',
                    'added relevance language rating': 'z',
                    'top added relevance': 'aa',
                    # 'add relevance task (seniors)': 'AB',
                    # 'full add relevance task (seniors)': 'AC',
                }
        print(f'Merged DataFrame shape: {new_results.shape}')
        print('\nColumns before adding empty columns:', [column for column in new_results.columns])
        print('Inserting empty columns...', end='\n\t')
        spreadsheet_columns = [letter for letter in string.ascii_uppercase]+['A'+letter for letter in string.ascii_uppercase]
        alphabet_dict = {char:idx for idx, char in enumerate(spreadsheet_columns)}
        for column_name, column_number in empty_columns.items():
            empty_column_loc = alphabet_dict[empty_columns[column_name].upper()] -1
            new_results.insert(loc=empty_column_loc, column=column_name, value='')
            print(f'{empty_columns[column_name].upper()} ({empty_column_loc}): {column_name}', end=', ')
        new_results.columns = [
            f'{spreadsheet_columns[index+1]}: {column}' for index, column in enumerate(new_results.columns)
            ]

    print(f'\n** Merged dataframe shape:', new_results.shape)
    print([column for column in new_results.columns])
    chain_results_dict[iteration_id] = new_results
    try:
        original_summary_time = next(iter(chatbot_dict[iteration_id].values())).date_created
    except:
        original_summary_time = 'previous_summaries'
    print(f'Original summary time: {original_summary_time}')
    if save_df:
        try:
            save_output(
                chain_results_dict[iteration_id], description=f'batch_Chaining_attributes_{original_summary_time}_append_{results_type}',
                csv_path=csv_path, pickle_path=pickle_path)
            print('')
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} DataFrame')
    if save_chatbot:
        try:
            print('Saving Chaining object (chatbot)...')
            save_instance_to_dict(
                chatbot_dict[iteration_id], 
                description=f'batch_Chaining_attributes_{original_summary_time}_append_{results_type}',
                pickle_path=pickle_path, json_path=json_path
                )
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} chatbot')
            
    return chain_results_dict

chain_results_dict = dict()
iteration_id = 1

chain_results_dict = merge_chaining_results(
    chain_results_dict, chatbot_dict, iteration_id, pivot=True,
    empty_columns=True
    )
# chain_results_dict[iteration_id].head()
# chain_results_dict[iteration_id].filter(regex='article|summary|statement|full')


Processing text1_prompt00...
No previous simple prompts for text1_prompt00
No previous relevance prompts for text1_prompt00
Processing text1_prompt01...
No previous simple prompts for text1_prompt01
No previous relevance prompts for text1_prompt01
Processing text1_prompt02...
No previous simple prompts for text1_prompt02
No previous relevance prompts for text1_prompt02
Processing text2_prompt00...
No previous simple prompts for text2_prompt00
No previous relevance prompts for text2_prompt00
Processing text2_prompt01...
No previous simple prompts for text2_prompt01
No previous relevance prompts for text2_prompt01
Processing text2_prompt02...
No previous simple prompts for text2_prompt02
No previous relevance prompts for text2_prompt02
Original summaries DataFrame shape: (30, 9)
Original summaries Dataframe columns: Index(['choice', 'article_title', 'system_role', 'model', 'text', 'prep step',
       'summarization task', 'full summarization task', 'summary'],
      dtype='object')
Simpl

In [149]:
chain_results_dict[iteration_id].head()

,B: choice,C: article_title,D: system_role,E: model,F: text,G: prep step,H: summarization task,I: full summarization task,J: summary,K: original summary content rating,L: original summary language rating,M: top summary,N: simple summary choice,O: audience,P: simplify task,Q: full simplify task,R: simple summary,S: simple summary content rating,T: simple summary language rating,U: top simple summary,V: relevance task,W: full relevance task,X: people who enjoy sports,Y: added relevance content rating,Z: added relevance language rating,AA: top added relevance,AB: add relevance task (seniors),AC: full add relevance task (seniors),AD: seniors
0,0,The Impact of Sleep Duration on Performance Among Competitive Athletes: A Systematic Literature ...,You are an expert at science communication.,gpt-3.5-turbo,The Impact of Sleep Duration on Performance Among Competitive Athletes: A Systematic Literature ...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post.,Take the key points and numerical descriptors to Summarize for a LinkedIn post.,A systematic literature review found no negative association between sleep duration and athletic...,,,,1,people without a science background,"If needed, rewrite the text using terms appropriate for the audience. If not keep it the same. ...","If needed, rewrite the text using terms appropriate for the audience. If not keep it the same. ...",A recent study found that getting enough sleep doesn't negatively affect how well athletes perfo...,,,,Rewrite this summary to include a statement of how it is relevant for the audience. Foll...,Rewrite this summary to include a statement of how it is relevant for the audience. Foll...,"Getting enough sleep is important for athletic performance, especially for aerobic sports. A rev...",,,,Rewrite this summary to include a statement of how it is relevant for the audience. Foll...,Rewrite this summary to include a statement of how it is relevant for the audience. Foll...,"A recent study found that getting enough sleep is important for athletic performance, especially..."
1,1,The Impact of Sleep Duration on Performance Among Competitive Athletes: A Systematic Literature ...,You are an expert at science communication.,gpt-3.5-turbo,The Impact of Sleep Duration on Performance Among Competitive Athletes: A Systematic Literature ...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post.,Take the key points and numerical descriptors to Summarize for a LinkedIn post.,"""Sleep duration has a positive impact on athletic performance, according to a systematic literat...",,,,1,people without a science background,"If needed, rewrite the text using terms appropriate for the audience. If not keep it the same. ...","If needed, rewrite the text using terms appropriate for the audience. If not keep it the same. ...","""Getting enough sleep can help you perform better in sports, according to a review of many studi...",,,,Rewrite this summary to include a statement of how it is relevant for the audience. Foll...,Rewrite this summary to include a statement of how it is relevant for the audience. Foll...,"Getting enough sleep can positively impact athletic performance, according to a review of publis...",,,,Rewrite this summary to include a statement of how it is relevant for the audience. Foll...,Rewrite this summary to include a statement of how it is relevant for the audience. Foll...,"Sleeping well can help seniors improve their athletic performance, according to a review of publ..."
2,2,The Impact of Sleep Duration on Performance Among Competitive Athletes: A Systematic Literature ...,You are an expert at science communication.,gpt-3.5-turbo,The Impact of Sleep Duration on Performance Among Competitive Athletes: A Systematic Literature ...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post.,Take the key points and numerical descriptors to Summarize for a LinkedIn post.,"""New literature review shows that incr

In [146]:
chain_results_dict[iteration_id].filter(regex='relevance task').head(1)

,V: relevance task,W: full relevance task,AB: add relevance task (seniors),AC: full add relevance task (seniors)
0,"Rewrite this summary to include a statement of how it is relevant for the audience. Follow these steps to accomplish this: \n1. Think about why this might be relevant for the audience in the grand scheme of things. \n2. If it is not evident why the text is relevant for the audience in the grand scheme of things, add a sentence to inform the audience. Otherwise, keep it the same. \n3. Modify the summary if needed to reduce redundancy. \n4. Check if the content and language are appropriate for the audience. If it is suitable for the audience, keep it the same. If not, rewrite using terms appropriate for the audience. \ \n \n5. Return the final version of the summary to be shown to the audience. \n6. Remove the backticks.\n \n\nYour audience consists of","Rewrite this summary to include a statement of how it is relevant for the audience. Follow these steps to accomplish this: \n1. Think about why this might be relevant for the audience in the grand scheme of things. \n2. If it is not evident why the text is relevant for the audience in the grand scheme of things, add a sentence to inform the audience. Otherwise, keep it the same. \n3. Modify the summary if needed to reduce redundancy. \n4. Check if the content and language are appropriate for the audience. If it is suitable for the audience, keep it the same. If not, rewrite using terms appropriate for the audience. \ \n \n5. Return the final version of the summary to be shown to the audience. \n6. Remove the backticks.\n \n\nYour audience consists of people who enjoy sports","Rewrite this summary to include a statement of how it is relevant for the audience. Follow these steps to accomplish this: \n1. Think about why this might be relevant for the audience in the grand scheme of things. \n2. If it is not evident why the text is relevant for the audience in the grand scheme of things, add a sentence to inform the audience. Otherwise, keep it the same. \n3. Modify the summary if needed to reduce redundancy. \n4. Check if the content and language are appropriate for the audience. If it is suitable for the audience, keep it the same. If not, rewrite using terms appropriate for the audience. \ \n \n5. Return the final version of the summary to be shown to the audience. \n6. Remove the backticks.\n \n\nYour audience consists of","Rewrite this summary to include a statement of how it is relevant for the audience. Follow these steps to accomplish this: \n1. Think about why this might be relevant for the audience in the grand scheme of things. \n2. If it is not evident why the text is relevant for the audience in the grand scheme of things, add a sentence to inform the audience. Otherwise, keep it the same. \n3. Modify the summary if needed to reduce redundancy. \n4. Check if the content and language are appropriate for the audience. If it is suitable for the audience, keep it the same. If not, rewrite using terms appropriate for the audience. \ \n \n5. Return the final version of the summary to be shown to the audience. \n6. Remove the backticks.\n \n\nYour audience consists of seniors"


### test without pivot

In [150]:
chain_results_dict = merge_chaining_results(
    chain_results_dict, chatbot_dict, iteration_id, pivot=False,
    empty_columns=True
    )

Processing text1_prompt00...
No previous simple prompts for text1_prompt00
No previous relevance prompts for text1_prompt00
Processing text1_prompt01...
No previous simple prompts for text1_prompt01
No previous relevance prompts for text1_prompt01
Processing text1_prompt02...
No previous simple prompts for text1_prompt02
No previous relevance prompts for text1_prompt02
Processing text2_prompt00...
No previous simple prompts for text2_prompt00
No previous relevance prompts for text2_prompt00
Processing text2_prompt01...
No previous simple prompts for text2_prompt01
No previous relevance prompts for text2_prompt01
Processing text2_prompt02...
No previous simple prompts for text2_prompt02
No previous relevance prompts for text2_prompt02
Original summaries DataFrame shape: (30, 9)
Original summaries Dataframe columns: Index(['choice', 'article_title', 'system_role', 'model', 'text', 'prep step',
       'summarization task', 'full summarization task', 'summary'],
      dtype='object')
Simpl

KeyError: "['audience', 'people who enjoy sports', 'seniors'] not in index"

## Iteration 6

In [202]:
import string
def merge_chaining_results(
        chain_results_dict, chatbot_dict, iteration_id, 
        empty_columns=None, pivot=True, validate=None,
        chatbot_id=None, save_df=False, save_chatbot=False, 
        csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
        pickle_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles',
        json_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\json'
        ):
    """
    Create a dataframe of new 'simple' or 'relevance' summaries from a Chaining object.
    Merge it with the original summaries DataFrame.

    Parameters:
        - chain_results_dict (dict): dictionary of DataFrames.
        - qna_dict (dict): dictionary of QnA DataFrames.
        - chatbot_dict (dict): dictionary of Chaining objects.
        - iteration_id (int, float, or string): iteration_id (dict key) of the chatbot_dict to process.
        - results_type (str): 'simple' or 'relevance'. Default is 'simple'.
        - empty_columns (Bool, int, or dict): dictionary of empty columns to add to the DataFrame. 
            If True or 1, default dictionary is used.
            If False or 0, no empty columns are added.
        - pivot (Bool): whether to pivot the relevance summaries DataFrame. Default is True.
        - validate (str): Argument to pass to pd.merge() to validate the merge.
        - chatbot_id (int, float, or string): chatbot_id (dict key) of the chatbot_dict to process.
        - save_df, save_chatbot (Bool): whether to save the DataFrame and chatbot_dict.

    """
    df_list_simple = []
    df_list_relevance = []
    qna_dfs_list = []
    iteration_id = chatbot_id if chatbot_id != None else iteration_id
    for chatbot_key in chatbot_dict[iteration_id].keys():
        print(f'Processing {chatbot_key}...')
        try: 
            n_previous_prompts_simple = chatbot_dict[iteration_id][chatbot_key].previous_n_prompts['simple']
            print(f'\tNumber of previous relevance prompts: {n_previous_prompts_simple}')
        except:
            n_previous_prompts_simple = 0
            print(f'No previous simple prompts for {chatbot_key}')
        try: 
            n_previous_prompts_relevance = chatbot_dict[iteration_id][chatbot_key].previous_n_prompts['relevance']
            print(f'\tNumber of previous relevance prompts: {n_previous_prompts_relevance}')
        except:
            n_previous_prompts_relevance = 0
            print(f'No previous relevance prompts for {chatbot_key}')
            
        # qna_dfs_list.append(pd.DataFrame(chatbot_dict[iteration_id][chatbot_key].qna))
        # qna_dfs_list.append(pd.DataFrame(chatbot_dict[iteration_id][chatbot_key].qna).reset_index(inplace=True, names=['choice']))
        qna_dfs_list.append(pd.DataFrame(chatbot_dict[iteration_id][chatbot_key].qna).reset_index(names=['choice']))

        # create results dictionaries that only grabs the results of the new prompts instead of all
        results_dict_simple = dict()
        total_n_prompts_simple = len(chatbot_dict[iteration_id][chatbot_key].simple_summary_dict)
        for prompt_number in range(n_previous_prompts_simple+1, total_n_prompts_simple+1):
            results_dict_simple[prompt_number] = chatbot_dict[iteration_id][chatbot_key].simple_summary_dict[prompt_number]
        chatbot_dict[iteration_id][chatbot_key].simple_summary_dict

        results_dict_relevance = dict()
        total_n_prompts_relevance = len(chatbot_dict[iteration_id][chatbot_key].relevance_dict)
        for prompt_number_relevance in range(n_previous_prompts_relevance+1, total_n_prompts_relevance+1):
            # print(f'\tAppending results for prompt {prompt_number_relevance} of {total_n_prompts_relevance}')
            results_dict_relevance[prompt_number_relevance] = chatbot_dict[iteration_id][chatbot_key].relevance_dict[prompt_number_relevance]

        for iteration_key_simple in results_dict_simple.keys():
            response_keys_simple = sorted([text_prompt_key for text_prompt_key in results_dict_simple[iteration_key_simple].keys()])
            # print(f'\tAppending results for {iteration_key_simple}: ', end='')

            for response_key_simple in response_keys_simple:
                df_list_simple.append(pd.DataFrame(results_dict_simple[iteration_key_simple][response_key_simple]).transpose())
                # print(f'{response_key_simple}, ', end='')
            # print('')
        for iteration_key_relevance in results_dict_relevance.keys():
            response_keys_relevance = sorted([text_prompt_key for text_prompt_key in results_dict_relevance[iteration_key_relevance].keys()])
            for response_key_relevance in response_keys_relevance:
                df_list_relevance.append(pd.DataFrame(results_dict_relevance[iteration_key_relevance][response_key_relevance]).transpose())
                # print(f'{response_key_relevance}, ', end='')
            # print('')

    
    simple_summary_df = pd.concat(df_list_simple)
    relevance_df = pd.concat(df_list_relevance)
    qna_df = pd.concat(qna_dfs_list)
    print(f'Original summaries DataFrame shape: {qna_df.shape}')
    print(f'Original summaries Dataframe columns: {qna_df.columns}')
    print('Simple summaries DataFrame shape:', simple_summary_df.shape)
    print(f'\tSimple summaries DataFrame columns: {[col for col in simple_summary_df.columns]}')
    print('Relevance summaries DataFrame shape:', relevance_df.shape)
    print(f'\tRelevance summaries DataFrame columns: {[col for col in relevance_df.columns]}')

    relevance_audience_list = sorted(relevance_df.audience.unique().tolist())
    print(f'Unique relevance audience values: {relevance_audience_list}')
    
    
    # common_columns = list(set(simple_summary_df.columns) & set(relevance_df.columns))
    # common_columns.remove('audience')


    new_results = qna_df.merge(
        simple_summary_df, how='right',
        right_on='original summary',
        left_on='summary',
        validate=validate
        ).drop(columns='original summary')
    if pivot == False:
        spreadsheet_columns = [
            "choice",
            "article_title",
            "system_role",
            "model",
            "text",
            "prep step",
            "summarization task",
            "full summarization task",
            "summary",
            "simple summary choice",
            "audience simplify",
            "simplify task",
            "full simplify task",
            "simple summary",
            "audience relevance",
            "relevance task",
            "full relevance task",
            "relevance statement"
        ]  

        validate=None
        
        print(f'DataFrame shape after merging with simple summaries: {new_results.shape}')
        print(f'\tColumns after merging with simple summaries: {[col for col in new_results.columns]}')
        new_results= new_results.merge(
            relevance_df, how='outer', suffixes=(' simplify', ' relevance'),
            left_on='summary', right_on='preceding summary', validate=validate
            ).drop(columns='preceding summary')
        # spreadsheet_columns.remove('audience')
    else:
        spreadsheet_columns = [
            "choice",
            "article_title",
            "system_role",
            "model",
            "text",
            "prep step",
            "summarization task",
            "full summarization task",
            "summary",
            "simple summary choice",
            "audience",
            "simplify task",
            "full simplify task",
            "simple summary",
            "relevance task",
            "full relevance task"
        ] 
        relevance_pivot_df = relevance_df.pivot(
            columns=['audience'],
            values='relevance statement',
            index=['preceding summary', 'relevance task',]
        ).sort_index().reset_index()
        new_results = new_results.merge(
            relevance_pivot_df, how='outer', suffixes=(' simplify', ' relevance'),
            left_on='summary', right_on='preceding summary',
            validate='m:1' if validate else None
        ).drop(columns='preceding summary')
        new_results['full relevance task'] = new_results['relevance task'].apply(lambda x: f'{x} {relevance_audience_list[0]}')
        new_results['add relevance task (seniors)'] = new_results["relevance task"]
        new_results['full add relevance task (seniors)'] =new_results['relevance task'].apply(lambda x: f'{x} {relevance_audience_list[1]}')
        spreadsheet_columns.append(relevance_audience_list[0])
        spreadsheet_columns.append('add relevance task (seniors)')
        spreadsheet_columns.append('full add relevance task (seniors)')
        spreadsheet_columns.append(relevance_audience_list[1])
        
    new_results = new_results[spreadsheet_columns]
    if empty_columns:
        if pivot == False:
            if (type(empty_columns) != dict):
                empty_columns = {
                    # "choice numnber": "C",
                    "original summary content rating": "K",
                    "original summary language rating": "L",
                    "top summary": "M",
                    # "simplify audience": "O",
                    # "simplify task": "P",
                    # "full simplify task": "Q",
                    # "simple summary": "R",
                    "simple summary content rating": "S",
                    "simple summary language rating": "T",
                    "top simple summary": "U",
                }
        else:           
            if (type(empty_columns) != dict):
                empty_columns = {
                    # "choice numnber": "C",
                    "original summary content rating": "K",
                    "original summary language rating": "L",
                    "top summary": "M",
                    # "simple summary choice": "N",
                    # "simplify audience": "O",
                    # "simplify task": "P",
                    # "full simplify task": "Q",
                    # "simple summary": "R",
                    "simple summary content rating": "S",
                    "simple summary language rating": "T",
                    'top simple summary': 'u',
                    # 'full add relevance task': 'w',
                    'added relevance content rating': 'y',
                    'added relevance language rating': 'z',
                    'top added relevance': 'aa',
                    # 'add relevance task (seniors)': 'AB',
                    # 'full add relevance task (seniors)': 'AC',
                }
        print(f'Merged DataFrame shape: {new_results.shape}')
        print('\nColumns before adding empty columns:', [column for column in new_results.columns])
        print('Inserting empty columns...', end='\n\t')
        spreadsheet_columns = [letter for letter in string.ascii_uppercase]+['A'+letter for letter in string.ascii_uppercase]
        alphabet_dict = {char:idx for idx, char in enumerate(spreadsheet_columns)}
        for column_name, column_number in empty_columns.items():
            empty_column_loc = alphabet_dict[empty_columns[column_name].upper()] -1
            new_results.insert(loc=empty_column_loc, column=column_name, value='')
            print(f'{empty_columns[column_name].upper()} ({empty_column_loc}): {column_name}', end=', ')
        new_results.columns = [
            f'{spreadsheet_columns[index+1]}: {column}' for index, column in enumerate(new_results.columns)
            ]

    print(f'\n** Merged dataframe shape:', new_results.shape)
    print([column for column in new_results.columns])
    chain_results_dict[iteration_id] = new_results
    try:
        original_summary_time = next(iter(chatbot_dict[iteration_id].values())).date_created
    except:
        original_summary_time = 'previous_summaries'
    print(f'Original summary time: {original_summary_time}')
    if save_df:
        try:
            save_output(
                chain_results_dict[iteration_id], description=f'batch_Chaining_attributes_{original_summary_time}_append_{results_type}',
                csv_path=csv_path, pickle_path=pickle_path)
            print('')
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} DataFrame')
    if save_chatbot:
        try:
            print('Saving Chaining object (chatbot)...')
            save_instance_to_dict(
                chatbot_dict[iteration_id], 
                description=f'batch_Chaining_attributes_{original_summary_time}_append_{results_type}',
                pickle_path=pickle_path, json_path=json_path
                )
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save {"simple summaries" if results_type=="simple" else "added relevance"} chatbot')
            
    return chain_results_dict

chain_results_dict = dict()
iteration_id = 1

chain_results_dict = merge_chaining_results(
    chain_results_dict, chatbot_dict, iteration_id, pivot=False,
    empty_columns=True
    )
# chain_results_dict[iteration_id]
# chain_results_dict[iteration_id].filter(regex='article|summary|statement|full')


Processing text1_prompt00...
No previous simple prompts for text1_prompt00
No previous relevance prompts for text1_prompt00
Processing text1_prompt01...
No previous simple prompts for text1_prompt01
No previous relevance prompts for text1_prompt01
Processing text1_prompt02...
No previous simple prompts for text1_prompt02
No previous relevance prompts for text1_prompt02
Processing text2_prompt00...
No previous simple prompts for text2_prompt00
No previous relevance prompts for text2_prompt00
Processing text2_prompt01...
No previous simple prompts for text2_prompt01
No previous relevance prompts for text2_prompt01
Processing text2_prompt02...
No previous simple prompts for text2_prompt02
No previous relevance prompts for text2_prompt02
Original summaries DataFrame shape: (30, 9)
Original summaries Dataframe columns: Index(['choice', 'article_title', 'system_role', 'model', 'text', 'prep step',
       'summarization task', 'full summarization task', 'summary'],
      dtype='object')
Simpl

In [199]:
chain_results_dict[iteration_id]

,B: choice,C: article_title,D: system_role,E: model,F: text,G: prep step,H: summarization task,I: full summarization task,J: summary,K: original summary content rating,L: original summary language rating,M: top summary,N: simple summary choice,O: audience simplify,P: simplify task,Q: full simplify task,R: simple summary,S: simple summary content rating,T: simple summary language rating,U: top simple summary,V: audience relevance,W: relevance task,X: full relevance task,Y: relevance statement
0,0,The Impact of Sleep Duration on Performance Am...,You are an expert at science communication.,gpt-3.5-turbo,The Impact of Sleep Duration on Performance Am...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post.,Take the key points and numerical descriptors ...,A systematic literature review found no negati...,,,,1,people without a science background,"If needed, rewrite the text using terms approp...","If needed, rewrite the text using terms approp...",A recent study found that getting enough sleep...,,,,seniors,Rewrite this summary to include a statement of...,Rewrite this summary to include a statement of...,A recent study found that getting enough sleep...
1,0,The Impact of Sleep Duration on Performance Am...,You are an expert at science communication.,gpt-3.5-turbo,The Impact of Sleep Duration on Performance Am...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post.,Take the key points and numerical descriptors ...,A systematic literature review found no negati...,,,,1,people without a science background,"If needed, rewrite the text using terms approp...","If needed, rewrite the text using terms approp...",A recent study found that getting enough sleep...,,,,people who enjoy sports,Rewrite this summary to include a statement of...,Rewrite this summary to include a statement of...,Getting enough sleep is important for athletic...
2,1,The Impact of Sleep Duration on Performance Am...,You are an expert at science communication.,gpt-3.5-turbo,The Impact of Sleep Duration on Performance Am...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post.,Take the key points and numerical descriptors ...,"""Sleep duration has a positive impact on athle...",,,,1,people without a science background,"If needed, rewrite the text using terms approp...","If needed, rewrite the text using terms approp...","""Getting enough sleep can help you perform bet...",,,,seniors,Rewrite this summary to include a statement of...,Rewrite this summary to include a statement of...,Sleeping well can help seniors improve their a...
3,1,The Impact of Sleep Duration on Performance Am...,You are an expert at science communication.,gpt-3.5-turbo,The Impact of Sleep Duration on Performance Am...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post.,Take the key points and numerical descriptors ...,"""Sleep duration has a positive impact on athle...",,,,1,people without a science background,"If needed, rewrite the text using terms approp...","If needed, rewrite the text using terms approp...","""Getting enough sleep can help you perform bet...",,,,people who enjoy sports,Rewrite this summary to include a statement of...,Rewrite this summary to include a statement of...,Getting enough sleep can positively impact ath...
4,2,The Impact of Sleep Duration on Performance Am...,You are an expert at science communication.,gpt-3.5-turbo,The Impact of Sleep Duration on Performance Am...,Take the key points and numerical descriptors to,Summarize for a LinkedIn post.,Take the key points and numerical descriptors ...,"""New literature review shows that increased sl...",,,,1,people without a science background,"If needed, rewrite the text using terms approp...","If needed, rewrite the text using terms approp...",A new study shows that getting more sleep can ...,,,,seniors,Rewrite this summary to include a statement of...,Rewrite this summary to include a statement of...,A new literature

# Save it

In [204]:
path

'../text/2023-05-03 4'

In [210]:
import string
def merge_chaining_results(
        chain_results_dict, chatbot_dict, iteration_id, 
        empty_columns=None, pivot=True, validate=None,
        chatbot_id=None, save_df=False, save_chatbot=False, 
        csv_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output',
        pickle_path=None,
        json_path=None
        ):
    """
    Create a dataframe of original, 'simple', 'relevance' summaries from a Chaining object.
    Merge it with the original summaries DataFrame.

    Parameters:
        - chain_results_dict (dict): dictionary of DataFrames.
        - chatbot_dict (dict): dictionary of Chaining objects.
        - iteration_id (int, float, or string): iteration_id (dict key) of the chatbot_dict to process.
        - empty_columns (Bool, int, or dict): dictionary of empty columns to add to the DataFrame. 
            If True or 1, default dictionary is used.
            If False or 0, no empty columns are added.
        - pivot (Bool): whether to pivot the relevance summaries DataFrame. Default is True.
        - validate (str): Argument to pass to pd.merge() to validate the merge.
        - chatbot_id (int, float, or string): chatbot_id (dict key) of the chatbot_dict to process.
        - save_df, save_chatbot (Bool): whether to save the DataFrame and chatbot_dict.
        - csv_path, pickle_path, and json_path (raw string or string): Location to save the 
            outputs. Must provide csv_path to save; pickle_path and json_path are optional and 
            default to the same as csv_path if not provided.
    """
    df_list_simple = []
    df_list_relevance = []
    qna_dfs_list = []
    iteration_id = chatbot_id if chatbot_id != None else iteration_id
    for chatbot_key in chatbot_dict[iteration_id].keys():
        print(f'Processing {chatbot_key}...')
        try: 
            n_previous_prompts_simple = chatbot_dict[iteration_id][chatbot_key].previous_n_prompts['simple']
            print(f'\tNumber of previous relevance prompts: {n_previous_prompts_simple}')
        except:
            n_previous_prompts_simple = 0
            print(f'No previous simple prompts for {chatbot_key}')
        try: 
            n_previous_prompts_relevance = chatbot_dict[iteration_id][chatbot_key].previous_n_prompts['relevance']
            print(f'\tNumber of previous relevance prompts: {n_previous_prompts_relevance}')
        except:
            n_previous_prompts_relevance = 0
            print(f'No previous relevance prompts for {chatbot_key}')
            
        # qna_dfs_list.append(pd.DataFrame(chatbot_dict[iteration_id][chatbot_key].qna))
        # qna_dfs_list.append(pd.DataFrame(chatbot_dict[iteration_id][chatbot_key].qna).reset_index(inplace=True, names=['choice']))
        qna_dfs_list.append(pd.DataFrame(chatbot_dict[iteration_id][chatbot_key].qna).reset_index(names=['choice']))

        # create results dictionaries that only grabs the results of the new prompts instead of all
        results_dict_simple = dict()
        total_n_prompts_simple = len(chatbot_dict[iteration_id][chatbot_key].simple_summary_dict)
        for prompt_number in range(n_previous_prompts_simple+1, total_n_prompts_simple+1):
            results_dict_simple[prompt_number] = chatbot_dict[iteration_id][chatbot_key].simple_summary_dict[prompt_number]
        chatbot_dict[iteration_id][chatbot_key].simple_summary_dict

        results_dict_relevance = dict()
        total_n_prompts_relevance = len(chatbot_dict[iteration_id][chatbot_key].relevance_dict)
        for prompt_number_relevance in range(n_previous_prompts_relevance+1, total_n_prompts_relevance+1):
            # print(f'\tAppending results for prompt {prompt_number_relevance} of {total_n_prompts_relevance}')
            results_dict_relevance[prompt_number_relevance] = chatbot_dict[iteration_id][chatbot_key].relevance_dict[prompt_number_relevance]

        for iteration_key_simple in results_dict_simple.keys():
            response_keys_simple = sorted([text_prompt_key for text_prompt_key in results_dict_simple[iteration_key_simple].keys()])
            # print(f'\tAppending results for {iteration_key_simple}: ', end='')

            for response_key_simple in response_keys_simple:
                df_list_simple.append(pd.DataFrame(results_dict_simple[iteration_key_simple][response_key_simple]).transpose())
                # print(f'{response_key_simple}, ', end='')
            # print('')
        for iteration_key_relevance in results_dict_relevance.keys():
            response_keys_relevance = sorted([text_prompt_key for text_prompt_key in results_dict_relevance[iteration_key_relevance].keys()])
            for response_key_relevance in response_keys_relevance:
                df_list_relevance.append(pd.DataFrame(results_dict_relevance[iteration_key_relevance][response_key_relevance]).transpose())
                # print(f'{response_key_relevance}, ', end='')
            # print('')

    
    simple_summary_df = pd.concat(df_list_simple)
    relevance_df = pd.concat(df_list_relevance)
    qna_df = pd.concat(qna_dfs_list)
    print(f'Original summaries DataFrame shape: {qna_df.shape}')
    print(f'Original summaries Dataframe columns: {qna_df.columns}')
    print('Simple summaries DataFrame shape:', simple_summary_df.shape)
    print(f'\tSimple summaries DataFrame columns: {[col for col in simple_summary_df.columns]}')
    print('Relevance summaries DataFrame shape:', relevance_df.shape)
    print(f'\tRelevance summaries DataFrame columns: {[col for col in relevance_df.columns]}')

    relevance_audience_list = sorted(relevance_df.audience.unique().tolist())
    print(f'Unique relevance audience values: {relevance_audience_list}')
    
    
    # common_columns = list(set(simple_summary_df.columns) & set(relevance_df.columns))
    # common_columns.remove('audience')


    new_results = qna_df.merge(
        simple_summary_df, how='right',
        right_on='original summary',
        left_on='summary',
        validate=validate
        ).drop(columns='original summary')
    if pivot == False:
        spreadsheet_columns = [
            "choice",
            "article_title",
            "system_role",
            "model",
            "text",
            "prep step",
            "summarization task",
            "full summarization task",
            "summary",
            "simple summary choice",
            "audience simplify",
            "simplify task",
            "full simplify task",
            "simple summary",
            "audience relevance",
            "relevance task",
            "full relevance task",
            "relevance statement"
        ]  

        validate=None
        
        print(f'DataFrame shape after merging with simple summaries: {new_results.shape}')
        print(f'\tColumns after merging with simple summaries: {[col for col in new_results.columns]}')
        new_results= new_results.merge(
            relevance_df, how='outer', suffixes=(' simplify', ' relevance'),
            left_on='summary', right_on='preceding summary', validate=validate
            ).drop(columns='preceding summary')
        # spreadsheet_columns.remove('audience')
    else:
        spreadsheet_columns = [
            "choice",
            "article_title",
            "system_role",
            "model",
            "text",
            "prep step",
            "summarization task",
            "full summarization task",
            "summary",
            "simple summary choice",
            "audience",
            "simplify task",
            "full simplify task",
            "simple summary",
            "relevance task",
            "full relevance task"
        ] 
        relevance_pivot_df = relevance_df.pivot(
            columns=['audience'],
            values='relevance statement',
            index=['preceding summary', 'relevance task',]
        ).sort_index().reset_index()
        new_results = new_results.merge(
            relevance_pivot_df, how='outer', suffixes=(' simplify', ' relevance'),
            left_on='summary', right_on='preceding summary',
            validate='m:1' if validate else None
        ).drop(columns='preceding summary')
        new_results['full relevance task'] = new_results['relevance task'].apply(lambda x: f'{x} {relevance_audience_list[0]}')
        new_results['add relevance task (seniors)'] = new_results["relevance task"]
        new_results['full add relevance task (seniors)'] =new_results['relevance task'].apply(lambda x: f'{x} {relevance_audience_list[1]}')
        spreadsheet_columns.append(relevance_audience_list[0])
        spreadsheet_columns.append('add relevance task (seniors)')
        spreadsheet_columns.append('full add relevance task (seniors)')
        spreadsheet_columns.append(relevance_audience_list[1])
        
    new_results = new_results[spreadsheet_columns]
    if empty_columns:
        if pivot == False:
            if (type(empty_columns) != dict):
                empty_columns = {
                    # "choice numnber": "C",
                    "original summary content rating": "K",
                    "original summary language rating": "L",
                    "top summary": "M",
                    # "simplify audience": "O",
                    # "simplify task": "P",
                    # "full simplify task": "Q",
                    # "simple summary": "R",
                    "simple summary content rating": "S",
                    "simple summary language rating": "T",
                    "top simple summary": "U",
                }
        else:           
            if (type(empty_columns) != dict):
                empty_columns = {
                    # "choice numnber": "C",
                    "original summary content rating": "K",
                    "original summary language rating": "L",
                    "top summary": "M",
                    # "simple summary choice": "N",
                    # "simplify audience": "O",
                    # "simplify task": "P",
                    # "full simplify task": "Q",
                    # "simple summary": "R",
                    "simple summary content rating": "S",
                    "simple summary language rating": "T",
                    'top simple summary': 'u',
                    # 'full add relevance task': 'w',
                    'added relevance content rating': 'y',
                    'added relevance language rating': 'z',
                    'top added relevance': 'aa',
                    # 'add relevance task (seniors)': 'AB',
                    # 'full add relevance task (seniors)': 'AC',
                }
        print(f'Merged DataFrame shape: {new_results.shape}')
        print('\nColumns before adding empty columns:', [column for column in new_results.columns])
        print('Inserting empty columns...', end='\n\t')
        spreadsheet_columns = [letter for letter in string.ascii_uppercase]+['A'+letter for letter in string.ascii_uppercase]
        alphabet_dict = {char:idx for idx, char in enumerate(spreadsheet_columns)}
        for column_name, column_number in empty_columns.items():
            empty_column_loc = alphabet_dict[empty_columns[column_name].upper()] -1
            new_results.insert(loc=empty_column_loc, column=column_name, value='')
            print(f'{empty_columns[column_name].upper()} ({empty_column_loc}): {column_name}', end=', ')
        new_results.columns = [
            f'{spreadsheet_columns[index+1]}: {column}' for index, column in enumerate(new_results.columns)
            ]

    print(f'\n** Merged dataframe shape:', new_results.shape)
    print([column for column in new_results.columns])
    chain_results_dict[iteration_id] = new_results
    try:
        original_summary_time = next(iter(chatbot_dict[iteration_id].values())).date_created
        description_tag = f'_{original_summary_time}_updated'
    except:
        description_tag=''
    print(f'Original summary time: {original_summary_time}')
    if save_df:
        pickle_path = csv_path if pickle_path is None else pickle_path
        try:
            save_output(
                chain_results_dict[iteration_id], 
                description=f'batch_Chaining_summaries{description_tag}',
                csv_path=csv_path, pickle_path=pickle_path)
            print('')
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save DataFrame')
    if save_chatbot:
        json_path = csv_path if json_path is None else json_path
        try:
            print('Saving Chaining object (chatbot)...')
            save_instance_to_dict(
                chatbot_dict[iteration_id], 
                description=f'batch_Chaining_attributes{description_tag}',
                pickle_path=pickle_path, json_path=json_path
                )
        except Exception as error:
            exc_type, exc_obj, tb = sys.exc_info()
            f = tb.tb_frame
            lineno = tb.tb_lineno
            filename = f.f_code.co_filename
            print("An error occurred on line", lineno, "in", filename, ":", error)
            print(f'Unable to save chatbot')
            
    return chain_results_dict

chain_results_dict = dict()
iteration_id = 1

# chain_results_dict = merge_chaining_results(
#     chain_results_dict, chatbot_dict, iteration_id, pivot=False,
#     empty_columns=True
#     )
    



chain_results_dict = merge_chaining_results(
    chain_results_dict, chatbot_dict, iteration_id, pivot=True,
    empty_columns=True,
    save_df=True, save_chatbot=True, 
        csv_path=path,
        # pickle_path=path,
        # json_path=path
    )


Processing text1_prompt00...
No previous simple prompts for text1_prompt00
No previous relevance prompts for text1_prompt00
Processing text1_prompt01...
No previous simple prompts for text1_prompt01
No previous relevance prompts for text1_prompt01
Processing text1_prompt02...
No previous simple prompts for text1_prompt02
No previous relevance prompts for text1_prompt02
Processing text2_prompt00...
No previous simple prompts for text2_prompt00
No previous relevance prompts for text2_prompt00
Processing text2_prompt01...
No previous simple prompts for text2_prompt01
No previous relevance prompts for text2_prompt01
Processing text2_prompt02...
No previous simple prompts for text2_prompt02
No previous relevance prompts for text2_prompt02
Original summaries DataFrame shape: (30, 9)
Original summaries Dataframe columns: Index(['choice', 'article_title', 'system_role', 'model', 'text', 'prep step',
       'summarization task', 'full summarization task', 'summary'],
      dtype='object')
Simpl

# Pandas display

In [162]:
# pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_colwidth', 50)


# *End of Page*